In [1]:
"""
Created on October 14 2018
@author: Neven Caplar
@contact: ncaplar@princeton.edu
"""
"""
Edited and remade on June 17th, 2019
@author: Theodore Pena
@contact: theodore.pena@tufts.edu
"""
print()

We're starting everything off with some basic imports.

In [2]:
import sys
print("sys version: {}".format(sys.version))

import matplotlib
import matplotlib.pyplot as plt
%matplotlib qt 
# If you don't have an xserver, the above line might crash your kernel. Try '%matplotlib inline' instead.

import numpy as np
print("numpy version: {}".format(np.__version__))

import pandas as pd
print("pandas version: {}".format(pd.__version__))

from tqdm import tqdm
# This gives for loops progress bars.

from sklearn.metrics.pairwise import euclidean_distances
# We need this to match AGN from one catalog to another.

from IPython.core.display import display, HTML
# An alternate, cleaner take on the jupyter workspace
display(HTML("<style>.container { width:100% !important; }</style>"))

sys version: 3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]
numpy version: 1.16.2
pandas version: 0.24.2


The below cell is just code prettification. Feel free to skip if you dislike any changes.

In [3]:
%%javascript
try {
  require(['base/js/utils'], function (utils) {
    utils.load_extension('code_prettify/code_prettify');
    utils.load_extension('collapsible_headings/main'); 
    utils.load_extension('codefolding/edit'); 
    utils.load_extension('codefolding/main'); 
    utils.load_extension('execute_time/ExecuteTime');   
    utils.load_extension('toc2/main'); 
  });
}
catch (err) {
  console.log('toc2 load error:', err);
}

<IPython.core.display.Javascript object>

Last thing we've got left is to define a few global variables we're going to come back to.

In [4]:
DATA_DIRECTORY='/home/tpena01/Variability/HSC/' 
# Change '/home/tpena01' to a path leading to Neven Caplar's Variability repo, presuming you've alredy cloned it.
# If not, here's a link: https://github.com/nevencaplar/Variability

# Data intake

## SDSS

The first catalog we'll need is the SDSS quasar catalog, DR7 edition. Read about it here: https://classic.sdss.org/dr7/products/value_added/qsocat_dr7.html.

In [5]:
dr7 = pd.read_csv(DATA_DIRECTORY+'dr7qso.dat', skiprows=80, sep='\s+', engine='python', error_bad_lines=False, header=None)

Skipping line 305: Expected 75 fields in line 305, saw 76. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 1633: Expected 75 fields in line 1633, saw 76. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 1745: Expected 75 fields in line 1745, saw 76. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 1828: Expected 75 fields in line 1828, saw 76. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 3631: Expected 75 fields in line 3631, saw 76. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 4123: Expected 75 fields in line 4123, saw 76. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 4124: Expected 75 fields in line 4124, saw 76. Error could possibly be due to quotes bei

Before we extract our list of positions, let's clean the catalog a bit.

In [128]:
dr7[dr7. != '1']

SyntaxError: invalid syntax (<ipython-input-128-dbd40ec5d6c1>, line 1)

In [121]:
test = dr7[ == 1]

SyntaxError: invalid syntax (<ipython-input-121-59179afcf8b7>, line 1)

In [8]:
Positions_of_dr7 = np.transpose([dr7[1].values, dr7[2].values])

## HSC

In [9]:
#extract ra and dec values for SQL query

data = np.zeros(len(Positions_of_dr7), dtype={'names':('ra', 'dec'),
                          'formats':('f8','f8')})

data['ra'] = Positions_of_dr7[:,0]
data['dec'] =  Positions_of_dr7[:,1]

PositionOfQuasars=data

np.save(DATA_DIRECTORY+'PositionOfQuasars',PositionOfQuasars)

In [10]:
PositionOfQuasar=np.load(DATA_DIRECTORY+'PositionOfQuasars.npy')

In [11]:
#############################
# This is how to find create values to be feed to SQL query system for HSC
# This query creates the table that we import in the analysis part below (so it can be skipped if you just want to just analyze result of this query).
# In terminal run command seen below - this is code which I modified from https://hsc-gitlab.mtk.nao.ac.jp/snippets/13
# Run the command in Python 2!!!
# the part in the command below that says <--columns ["g_cmodel_mag","g_cmodel_mag"]> does not mean I am just taking g_cmodel magnitude - I needed something as a filler

# run 'python CatalogCreator.py PositionOfQuasars.npy --columns ["g_cmodel_mag","g_cmodel_mag"]'

#############################

In [12]:
# This is example of the SQL query to find the data
# Copy paste result from the CatalogCreator.py (also avaliable on github) command into VALUES() bracket, make sure VALUES() is in only one line, and run query at https://hscdata.mtk.nao.ac.jp:4443/datasearch/
# This run took around 3 hours

'''
WITH
    user_catalog("user.ra","user.dec") AS (VALUES('2.7227999642500000e-02'::double precision,'5.1534098386799998e-01'::double precision),('3.3900000154999999e-02','2.7630099654200002e-01'),('3.8603998720599997e-02','1.5298477172900000e+01'),('3.9089001715200002e-02','1.3938449859600000e+01'),('3.9271000772700002e-02','-1.0464426040599999e+01'),('4.7548998147199997e-02','1.4929354667700000e+01'),('4.9839001148899997e-02','4.0364999324100002e-02'),('5.1079001277700001e-02','-5.3904700279200002e-01'),('5.4786998778600000e-02','1.4176302909900000e+01'),('5.7505998760499999e-02','-9.1300100088100000e-01'))
    ,
    match AS (
        SELECT
            object_id,
            earth_distance(coord, ll_to_earth("user.dec", "user.ra")) AS match_distance,
            user_catalog.*
        FROM
            user_catalog JOIN s18a_wide.forced
                ON coneSearch(coord, "user.ra", "user.dec", 0.1) 
    )
SELECT
	f.ra,
	f.dec,
	f.g_cmodel_mag,
	f.g_cmodel_magsigma,
	f.r_cmodel_mag,
	f.r_cmodel_magsigma,
	f.i_cmodel_mag,
	f.i_cmodel_magsigma,
    f2.g_psfflux_mag,
    f2.g_psfflux_magsigma,
    f2.r_psfflux_mag,
    f2.r_psfflux_magsigma,
    f2.i_psfflux_mag,
    f2.i_psfflux_magsigma,
    f.object_id
FROM
    match LEFT JOIN s18a_wide.forced AS f USING(object_id)
JOIN s18a_wide.forced2 AS f2 ON f.object_id = f2.object_id
WHERE
    f.isprimary
'''

'\nWITH\n    user_catalog("user.ra","user.dec") AS (VALUES(\'2.7227999642500000e-02\'::double precision,\'5.1534098386799998e-01\'::double precision),(\'3.3900000154999999e-02\',\'2.7630099654200002e-01\'),(\'3.8603998720599997e-02\',\'1.5298477172900000e+01\'),(\'3.9089001715200002e-02\',\'1.3938449859600000e+01\'),(\'3.9271000772700002e-02\',\'-1.0464426040599999e+01\'),(\'4.7548998147199997e-02\',\'1.4929354667700000e+01\'),(\'4.9839001148899997e-02\',\'4.0364999324100002e-02\'),(\'5.1079001277700001e-02\',\'-5.3904700279200002e-01\'),(\'5.4786998778600000e-02\',\'1.4176302909900000e+01\'),(\'5.7505998760499999e-02\',\'-9.1300100088100000e-01\'))\n    ,\n    match AS (\n        SELECT\n            object_id,\n            earth_distance(coord, ll_to_earth("user.dec", "user.ra")) AS match_distance,\n            user_catalog.*\n        FROM\n            user_catalog JOIN s18a_wide.forced\n                ON coneSearch(coord, "user.ra", "user.dec", 0.1) \n    )\nSELECT\n\tf.ra,\n\tf.dec

In [13]:
# This is the example of the SQL query to find the data, as ran on the server on February 8, 2019
# Added quality flags from Yusra AlSayyad
# Copy paste result from the CatalogCreator.py (also avaliable on github) command into VALUES() bracket, make sure VALUES() is in only one line, and run query at https://hscdata.mtk.nao.ac.jp:4443/datasearch/
# This run took around 3 hours

'''
WITH
    user_catalog("user.ra","user.dec") AS (VALUES('2.7227999642500000e-02'::double precision,'5.1534098386799998e-01'::double precision),('3.3900000154999999e-02','2.7630099654200002e-01'),('3.8603998720599997e-02','1.5298477172900000e+01'),('3.9089001715200002e-02','1.3938449859600000e+01'),('3.9271000772700002e-02','-1.0464426040599999e+01'),('4.7548998147199997e-02','1.4929354667700000e+01'),('4.9839001148899997e-02','4.0364999324100002e-02'),('5.1079001277700001e-02','-5.3904700279200002e-01'),('5.4786998778600000e-02','1.4176302909900000e+01'),('5.7505998760499999e-02','-9.1300100088100000e-01'))
    ,
    match AS (
        SELECT
            object_id,
            earth_distance(coord, ll_to_earth("user.dec", "user.ra")) AS match_distance,
            user_catalog.*
        FROM
            user_catalog JOIN s18a_wide.forced
                ON coneSearch(coord, "user.ra", "user.dec", 0.1) 
    )
SELECT
	f.ra,
	f.dec,
	f.g_cmodel_mag,
	f.g_cmodel_magsigma,
	f.r_cmodel_mag,
	f.r_cmodel_magsigma,
	f.i_cmodel_mag,
	f.i_cmodel_magsigma,
    f2.g_psfflux_mag,
    f2.g_psfflux_magsigma,
    f2.r_psfflux_mag,
    f2.r_psfflux_magsigma,
    f2.i_psfflux_mag,
    f2.i_psfflux_magsigma,
    f.object_id
    f.g_psfflux_flag
    f.r_psfflux_flag
    f.i_psfflux_flag
    f.g_cmodel_flag 
    f.r_cmodel_flag
    f.i_cmodel_flag
    f.g_pixelflags_edge
    f.r_pixelflags_edge
    f.i_pixelflags_edge        
    f.g_pixelflags_bad 
    f.r_pixelflags_bad
    f.i_pixelflags_bad
    f.g_pixelflags_interpolatedcenter
    f.r_pixelflags_interpolatedcenter
    f.i_pixelflags_interpolatedcenter
    f.g_pixelflags_saturatedcenter 
    f.r_pixelflags_saturatedcenter
    f.i_pixelflags_saturatedcenter
    f.g_pixelflags_crcenter
    f.r_pixelflags_crcenter
    f.i_pixelflags_crcenter
FROM
    match LEFT JOIN s18a_wide.forced AS f USING(object_id)
JOIN s18a_wide.forced2 AS f2 ON f.object_id = f2.object_id
WHERE
    f.isprimary

'''

'\nWITH\n    user_catalog("user.ra","user.dec") AS (VALUES(\'2.7227999642500000e-02\'::double precision,\'5.1534098386799998e-01\'::double precision),(\'3.3900000154999999e-02\',\'2.7630099654200002e-01\'),(\'3.8603998720599997e-02\',\'1.5298477172900000e+01\'),(\'3.9089001715200002e-02\',\'1.3938449859600000e+01\'),(\'3.9271000772700002e-02\',\'-1.0464426040599999e+01\'),(\'4.7548998147199997e-02\',\'1.4929354667700000e+01\'),(\'4.9839001148899997e-02\',\'4.0364999324100002e-02\'),(\'5.1079001277700001e-02\',\'-5.3904700279200002e-01\'),(\'5.4786998778600000e-02\',\'1.4176302909900000e+01\'),(\'5.7505998760499999e-02\',\'-9.1300100088100000e-01\'))\n    ,\n    match AS (\n        SELECT\n            object_id,\n            earth_distance(coord, ll_to_earth("user.dec", "user.ra")) AS match_distance,\n            user_catalog.*\n        FROM\n            user_catalog JOIN s18a_wide.forced\n                ON coneSearch(coord, "user.ra", "user.dec", 0.1) \n    )\nSELECT\n\tf.ra,\n\tf.dec

# Analysis of the output

## Initial Analysis

In [14]:
# This is dr7 qso catalog from https://classic.sdss.org/dr7/products/value_added/qsocat_dr7.html
dr7 = pd.read_csv(DATA_DIRECTORY+'dr7qso.dat',skiprows=80,sep='\s+',engine='python', error_bad_lines=False, header=None )
Positions_of_dr7=np.transpose([dr7[1].values,dr7[2].values])

Skipping line 305: Expected 75 fields in line 305, saw 76. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 1633: Expected 75 fields in line 1633, saw 76. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 1745: Expected 75 fields in line 1745, saw 76. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 1828: Expected 75 fields in line 1828, saw 76. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 3631: Expected 75 fields in line 3631, saw 76. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 4123: Expected 75 fields in line 4123, saw 76. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 4124: Expected 75 fields in line 4124, saw 76. Error could possibly be due to quotes bei

In [15]:
#read in catalogue from HSC query with pandas
#This is the version without quality flags
#df = pd.read_csv(DATA_DIRECTORY+'192565.csv')

#version with flags
df = pd.read_csv(DATA_DIRECTORY+'194782.csv')

# give ra and dec
Positions_of_df=np.transpose([df['# ra'].values,df['dec'].values])

In [16]:
# check the overlap with SDSS
# import dr7 (at the start of section 1), if you skipped section 1
%matplotlib qt
plt.scatter(Positions_of_dr7[:,0],Positions_of_dr7[:,1],color='black', s=20)
plt.scatter(Positions_of_df[:,0],Positions_of_df[:,1],color='red', s=5)
plt.show()

In [74]:
print("1")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>343.50) & (Positions_of_dr7[:,0]<343.60) & (Positions_of_dr7[:,1]>0.940) & (Positions_of_dr7[:,1]<0.95))])

print("2")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>345.05) & (Positions_of_dr7[:,0]<345.10) & (Positions_of_dr7[:,1]>0.71) & (Positions_of_dr7[:,1]<0.72))])

print("3")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>343.45) & (Positions_of_dr7[:,0]<343.60) & (Positions_of_dr7[:,1]>0.57) & (Positions_of_dr7[:,1]<0.58))])

print("4")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>344.90) & (Positions_of_dr7[:,0]<345.00) & (Positions_of_dr7[:,1]>0.50) & (Positions_of_dr7[:,1]<0.52))])

print("5")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>345.20) & (Positions_of_dr7[:,0]<345.40) & (Positions_of_dr7[:,1]>0.50) & (Positions_of_dr7[:,1]<0.52))])

print("6")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>345.60) & (Positions_of_dr7[:,0]<345.80) & (Positions_of_dr7[:,1]>0.37) & (Positions_of_dr7[:,1]<0.40))])

print("7")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>345.54) & (Positions_of_dr7[:,0]<345.57) & (Positions_of_dr7[:,1]>0.328) & (Positions_of_dr7[:,1]<0.330))])

print("8")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>345.27) & (Positions_of_dr7[:,0]<345.30) & (Positions_of_dr7[:,1]>0.322) & (Positions_of_dr7[:,1]<0.326))])

print("9")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>345.02) & (Positions_of_dr7[:,0]<345.05) & (Positions_of_dr7[:,1]>0.294) & (Positions_of_dr7[:,1]<0.296))])

print("10")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>344.20) & (Positions_of_dr7[:,0]<344.40) & (Positions_of_dr7[:,1]>0.18) & (Positions_of_dr7[:,1]<0.20))])

print("11")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>344.0) & (Positions_of_dr7[:,0]<344.10) & (Positions_of_dr7[:,1]>0.14) & (Positions_of_dr7[:,1]<0.16))])

print("12")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>345.19) & (Positions_of_dr7[:,0]<345.23) & (Positions_of_dr7[:,1]>0.17) & (Positions_of_dr7[:,1]<0.18))])

print("13")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>344.90) & (Positions_of_dr7[:,0]<345.10) & (Positions_of_dr7[:,1]>0.12) & (Positions_of_dr7[:,1]<0.13))])

print("14")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>343.80) & (Positions_of_dr7[:,0]<343.90) & (Positions_of_dr7[:,1]>-0.01) & (Positions_of_dr7[:,1]<0.01))])

print("15")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>344.5) & (Positions_of_dr7[:,0]<344.6) & (Positions_of_dr7[:,1]>-0.07) & (Positions_of_dr7[:,1]<-0.05))])

print("16")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>343.90) & (Positions_of_dr7[:,0]<344.0) & (Positions_of_dr7[:,1]>-0.16) & (Positions_of_dr7[:,1]<-0.14))])

print("17")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>345.24) & (Positions_of_dr7[:,0]<345.30) & (Positions_of_dr7[:,1]>-0.41) & (Positions_of_dr7[:,1]<-0.39))])

print("18")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>344.05) & (Positions_of_dr7[:,0]<344.10) & (Positions_of_dr7[:,1]>-0.57) & (Positions_of_dr7[:,1]<-0.55))])

print("19")
print(Positions_of_dr7[np.where((Positions_of_dr7[:,0]>344.85) & (Positions_of_dr7[:,0]<344.90) & (Positions_of_dr7[:,1]>-0.96) & (Positions_of_dr7[:,1]<-0.95))])

1
[[343.539981   0.941435]]
2
[[345.091915   0.716739]]
3
[[343.538706   0.574188]]
4
[[344.947254   0.50916 ]]
5
[[345.293804   0.515499]]
6
[[345.620658   0.380341]]
7
[[3.45548222e+02 3.28786000e-01]]
8
[[3.452765e+02 3.235220e-01]]
9
[[3.45030316e+02 2.94201000e-01]]
10
[[3.44369552e+02 1.89026000e-01]]
11
[[3.44077751e+02 1.48786000e-01]]
12
[[3.45207442e+02 1.75392000e-01]]
13
[[3.45002633e+02 1.26146000e-01]]
14
[[ 3.43892104e+02 -4.76100000e-03]]
15
[[ 3.44579177e+02 -6.44520000e-02]]
16
[[ 3.43960003e+02 -1.54925000e-01]]
17
[[345.274163  -0.400494]]
18
[[344.069482  -0.566076]]
19
[[344.870285  -0.95636 ]]


In [17]:
# takes a bit less than 1 minute on my laptop
# this matches QSO from SDSS and resulting catalog from HSC
res_matching=[]
for j in tqdm(range(len(Positions_of_dr7))):
    # finds distance from each of the objects in dr7 catalogue from the objects in HSC catalogue
    PositionOfQuasars_euclidean_distances=euclidean_distances([Positions_of_dr7[j]],Positions_of_df)
    # shortest distance
    shortest_distance=np.min(PositionOfQuasars_euclidean_distances[0])
    # element of the ``Positions_of_df'' that has the shortest distance to the SDSS QSO
    shortest_distance_index=np.where(PositionOfQuasars_euclidean_distances[0]==np.min(PositionOfQuasars_euclidean_distances[0]))[0][0]
    res_matching.append([shortest_distance,df.loc[shortest_distance_index].values])

100%|██████████| 105645/105645 [01:27<00:00, 1205.72it/s]


In [18]:
# create array that has objects from SDSS that are found in HSC

matched_array=[]
matched_array_extended_SDSS=[]
for i in tqdm(range(len(res_matching))):
    if res_matching[i][0]>0.001:
        pass
    else:
        # extract columns 0,1,2,3,6,7,8,9,10,11 from SDSS, which are SDSS ID, ra, dec, redshift and psf measurments in different bands (g->[6,7],r->[8,9],i->[10,11])
        matched_array.append(np.concatenate((dr7.loc[i][[0,1,2,3,6,7,8,9,10,11]],res_matching[i][1])))
        # extract columns 1,2,3 which are ra, dec and redshift (plan to add mass from Schen catalog)
        matched_array_extended_SDSS.append(np.concatenate((dr7.loc[i][[1,2,3]],res_matching[i][1])))
        
matched_array=np.array(matched_array)
matched_array_extended_SDSS=np.array(matched_array_extended_SDSS)

100%|██████████| 105645/105645 [00:18<00:00, 5794.61it/s] 


In [19]:
# filter out QSO which failed measurment, (g-band set at 0 in SDSS catlog)
matched_array_filtered=matched_array[matched_array[:,4]>16]
matched_array_extended_SDSS_filtered=matched_array_extended_SDSS[matched_array[:,4]>16]

In [20]:
# plot differences using cmodel values from HSC
plt.figure(figsize=(20,6))
plt.subplot(131)

plt.scatter(matched_array_filtered[:,4],matched_array_filtered[:,8+4],s=1)
plt.scatter(matched_array_filtered[:,4][(matched_array_filtered[:,8+4]-matched_array_filtered[:,4])>1],matched_array_filtered[:,8+4][(matched_array_filtered[:,8+4]-matched_array_filtered[:,4])>1],s=2)
plt.scatter(matched_array_filtered[:,4][(matched_array_filtered[:,8+4]-matched_array_filtered[:,4])>2],matched_array_filtered[:,8+4][(matched_array_filtered[:,8+4]-matched_array_filtered[:,4])>2],s=3,color='red')

plt.xlabel('SDSS g magnitude')
plt.ylabel('HSC cmodel g magnitude')
plt.plot(range(0,100),range(0,100),color='black',ls='--')
plt.xlim(17,25)
plt.ylim(17,25)

plt.subplot(132)

plt.scatter(matched_array_filtered[:,6],matched_array_filtered[:,8+6],s=1)
plt.scatter(matched_array_filtered[:,6][(matched_array_filtered[:,8+4]-matched_array_filtered[:,4])>1],matched_array_filtered[:,8+6][(matched_array_filtered[:,8+4]-matched_array_filtered[:,4])>1],s=2)
plt.scatter(matched_array_filtered[:,6][(matched_array_filtered[:,8+4]-matched_array_filtered[:,4])>2],matched_array_filtered[:,8+6][(matched_array_filtered[:,8+4]-matched_array_filtered[:,4])>2],s=3,color='red')
plt.xlabel('SDSS r magnitude')
plt.ylabel('HSC cmodel r magnitude')
plt.plot(range(0,100),range(0,100),color='black',ls='--')
plt.xlim(17,25)
plt.ylim(17,25)

plt.subplot(133)

plt.scatter(matched_array_filtered[:,8],matched_array_filtered[:,8+8],s=1)
plt.scatter(matched_array_filtered[:,8][(matched_array_filtered[:,8+4]-matched_array_filtered[:,4])>1],matched_array_filtered[:,8+8][(matched_array_filtered[:,8+4]-matched_array_filtered[:,4])>1],s=2)
plt.scatter(matched_array_filtered[:,8][(matched_array_filtered[:,8+4]-matched_array_filtered[:,4])>2],matched_array_filtered[:,8+8][(matched_array_filtered[:,8+4]-matched_array_filtered[:,4])>2],s=3,color='red')
plt.xlabel('SDSS i magnitude')
plt.ylabel('HSC cmodel i magnitude')
plt.plot(range(0,100),range(0,100),color='black',ls='--')
plt.xlim(17,25)
plt.ylim(17,25)
plt.show()

In [79]:
# plot differences using psf values from HSC
plt.figure(figsize=(20,6))
plt.subplot(131)

plt.scatter(matched_array_filtered[:,4],matched_array_filtered[:,14+4],s=1)
plt.scatter(matched_array_filtered[:,4][(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>1],matched_array_filtered[:,14+4][(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>1],s=2)
plt.scatter(matched_array_filtered[:,4][(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>2],matched_array_filtered[:,14+4][(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>2],s=3,color='red')

plt.xlabel('SDSS g magnitude')
plt.ylabel('HSC psf g magnitude')
plt.plot(range(0,100),range(0,100),color='black',ls='--')
plt.xlim(17,25)
plt.ylim(17,25)

plt.subplot(132)

plt.scatter(matched_array_filtered[:,6],matched_array_filtered[:,14+6],s=1)
plt.scatter(matched_array_filtered[:,6][(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>1],matched_array_filtered[:,14+6][(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>1],s=2)
plt.scatter(matched_array_filtered[:,6][(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>2],matched_array_filtered[:,14+6][(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>2],s=3,color='red')
plt.xlabel('SDSS r magnitude')
plt.ylabel('HSC psf r magnitude')
plt.plot(range(0,100),range(0,100),color='black',ls='--')
plt.xlim(17,25)
plt.ylim(17,25)
plt.subplot(133)

plt.scatter(matched_array_filtered[:,8],matched_array_filtered[:,14+8],s=1)
plt.scatter(matched_array_filtered[:,8][(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>1],matched_array_filtered[:,14+8][(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>1],s=2)
plt.scatter(matched_array_filtered[:,8][(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>2],matched_array_filtered[:,14+8][(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>2],s=3,color='red')
plt.xlabel('SDSS i magnitude')
plt.ylabel('HSC psf i magnitude')
plt.plot(range(0,100),range(0,100),color='black',ls='--')
plt.xlim(17,25)
plt.ylim(17,25)
plt.show()

## Adding MacLeod catalogue

In [ ]:
#########
#Can be skipped, just used for consistency check and to compare results
#########

#read in catalogue with pandas
MacLeod_Cat= np.loadtxt(DATA_DIRECTORY+'MacLeod2019_tab2.txt',dtype='str')

ra_dec_MacLeod_Cat_step1=[]
for i in range(len(MacLeod_Cat)):
    if '+' in MacLeod_Cat[:,0][i]:
        split_string=str.split(MacLeod_Cat[:,0][i],'+')
        split_string.insert(1,'+1')
        ra_dec_MacLeod_Cat_step1.append(split_string)
    else:
        split_string=str.split(MacLeod_Cat[:,0][i],'-')
        split_string.insert(1,'-1')
        ra_dec_MacLeod_Cat_step1.append(split_string)

ra_dec_MacLeod_Cat_step1=np.array(ra_dec_MacLeod_Cat_step1)

In [ ]:
# split strings from MacLeod catalogue 
ra_dec_MacLeod_Cat_step2=[]
n=2
for i in range(len(ra_dec_MacLeod_Cat_step1)):
    ra_string=ra_dec_MacLeod_Cat_step1[i][0]
    dec_string=ra_dec_MacLeod_Cat_step1[i][2]
    ra_string_split=[ra_string[i:i+n] for i in range(0, len(ra_string), n)]
    dec_string_split=[dec_string[i:i+n] for i in range(0, len(dec_string), n)]
    ra_split=[float(ra_string_split[0]),float(ra_string_split[1]),float(ra_string_split[2]),float(ra_string_split[3]),float(ra_string_split[4])]
    dec_split=[float(ra_dec_MacLeod_Cat_step1[0][1])*float(dec_string_split[0]),float(dec_string_split[1]),float(dec_string_split[2]),float(dec_string_split[3])]
    ra_dec_MacLeod_Cat_step2.append([ra_split,dec_split])

In [ ]:
# move in the same format as other catalogues
ra_dec_MacLeod_Cat_step3=[]
for i in range(len(ra_dec_MacLeod_Cat_step2)):
    ra_dec_MacLeod_Cat_step3.append([ra_dec_MacLeod_Cat_step2[i][0][0]*15+(ra_dec_MacLeod_Cat_step2[i][0][1]/60)*15+((ra_dec_MacLeod_Cat_step2[i][0][2]+ra_dec_MacLeod_Cat_step2[i][0][3])/3600)*15,
     ra_dec_MacLeod_Cat_step2[i][1][0]+(ra_dec_MacLeod_Cat_step2[i][1][1]/60)+(ra_dec_MacLeod_Cat_step2[i][1][2]/3600)])
    
ra_dec_MacLeod_Cat_step3=np.array(ra_dec_MacLeod_Cat_step3)

In [ ]:
# check the overlap with SDSS
%matplotlib qt
plt.figure(figsize=(12,8))
plt.scatter(Positions_of_dr7[:,0],Positions_of_dr7[:,1],color='black',label='SDSS')
plt.scatter(Positions_of_df[:,0],Positions_of_df[:,1],color='blue',label='HSC')
plt.scatter(ra_dec_MacLeod_Cat_step3[:,0],ra_dec_MacLeod_Cat_step3[:,1],color='red',label='MacLeod')
plt.legend()
plt.show()

In [ ]:
# again takes around a minute
res=[]
for j in tqdm(range(len(Positions_of_dr7))):
    
    #HSC catalogue search; same as above
    # finds distance from each of the objects in dr7 catalogue from the objects in HSC catalogue
    PositionOfQuasars_euclidean_distances=euclidean_distances([Positions_of_dr7[j]],Positions_of_df)
    # shortest distance
    shortest_distance_HSC=np.min(PositionOfQuasars_euclidean_distances[0])
    # element of the ``Positions_of_df'' that has the shortest distance to the SDSS QSO
    shortest_distance_index_HSC=np.where(PositionOfQuasars_euclidean_distances[0]==shortest_distance_HSC)[0][0]
    
    
    
    #MacLeod catalogue search; same as above
    # finds distance from each of the objects in dr7 catalogue from the objects in HSC catalogue
    PositionOfQuasars_euclidean_distances=euclidean_distances([Positions_of_dr7[j]],ra_dec_MacLeod_Cat_step3)
    # shortest distance
    shortest_distance_MacLeod=np.min(PositionOfQuasars_euclidean_distances[0])
    # element of the ``Positions_of_df'' that has the shortest distance to the SDSS QSO
    shortest_distance_index_MacLeod=np.where(PositionOfQuasars_euclidean_distances[0]==shortest_distance_MacLeod)[0][0]    
    
    res.append([shortest_distance_HSC,shortest_distance_MacLeod,df.loc[shortest_distance_index_HSC].values,MacLeod_Cat[shortest_distance_index_MacLeod]])

In [ ]:
# extract columns 0,1,2,3,6,7,8,9,10,11 from SDSS, which are as above: sdss id, ra, dec, z, and measurments in different bands (gri)
matched_array_MacLeod_SDSS=[]
not_matched_array_MacLeod_SDSS=[]
for i in tqdm(range(len(res))):
    # is object close to HSC 
    if res[i][0]>0.001:
        pass
    else:
        # if the objects is avaliabe in HSC, search for it in MacLeod
        if res[i][1]>0.001:
            pass
        else:
            # join the SDSSvalue, with the value from HSC and MacLeod
            matched_array_MacLeod_SDSS.append(np.concatenate((dr7.loc[i][[0,1,2,3,6,7,8,9,10,11]],res[i][2],res[i][3])))
        
matched_array_MacLeod_SDSS=np.array(matched_array_MacLeod_SDSS)

In [ ]:
# check the overlap of the matched_array_MacLeod_SDSS wit the SDSS
print('number of objects that have been found is: '+str(len(matched_array_MacLeod_SDSS)))
plt.figure(figsize=(10,6))
plt.scatter(Positions_of_dr7[:,0],Positions_of_dr7[:,1],color='black')
plt.scatter(matched_array_MacLeod_SDSS[:,1],matched_array_MacLeod_SDSS[:,2],color='red')
plt.show()

In [ ]:
# columns of matched_array_MacLeod_SDSS
# 0. SDSS ID, 1. ra (SDSS), 2. dec (SDSS), 3. redshift (SDSS), 4. g-mag (SDSS) 5. sigma g-mag (SDSS), 6. r-mag (SDSS) 7. sigma r-mag (SDSS), 8. i-mag (SDSS) 9. sigma i-mag (SDSS)
# 10. ra (HSC), 11. dec (HSC), 12. g_cmodel_mag, 13. g_cmodel_magsigma, 14. r_cmodel_mag, 15. r_cmodel_magsigma, 16. i_cmodel_mag, 17. i_cmodel_magsigma,
# 18. g_psfflux_mag, 19. g_psfflux_magsigma, 20. r_psfflux_mag, 21. r_psfflux_magsigma, 22. i_psfflux_mag, 23. i_psfflux_magsigma, 24. object_id
# 25. SDSSJID, 26. z, 27. morph. flag (=0 for point source, 1 for extended), 28. Phot. MJD_1, 29. g_1 (mag), 30. sigma_1 (mag), 31. Phot. MJD_2
# 32. g_2 (mag), 33. sigma_2 (mag), 34. Spec. MJD_1, 35.  MJD (PS1), 36. g_PS1 (mag), 37. sigma_PS1 (mag), 38. Spec. MJD_2, 39. Facility , 
# 40. CLQ by VI?  (=1 for visual CLQ, 0 otherwise), 41. N_sigma (Hbeta) 

In [ ]:
# Check that you selected correct objects by comparing SDSS and MacLeod catalogue
is_correctly_matched=[]
for i in range(len(matched_array_MacLeod_SDSS)):
    is_correctly_matched.append(matched_array_MacLeod_SDSS[i][0]==matched_array_MacLeod_SDSS[i][25])
    
# if result is 1, it is correct
print(np.mean(is_correctly_matched))

In [ ]:
mag_selection=(matched_array_filtered[:,2]<10)&(matched_array_filtered[:,1]<16*15)&(matched_array_filtered[:,1]>10*15)

In [ ]:
matplotlib.rcParams.update({'font.size': 16})

#showing the same plot, but now with the psf magnitudes
selection_of_objects_1_mag_dimmer=np.abs((matched_array_filtered[:,14+4]-matched_array_filtered[:,4]))>1
selection_of_objects_2_mag_dimmer=np.abs((matched_array_filtered[:,14+4]-matched_array_filtered[:,4]))>2

selection_of_objects_actual_1_mag_dimmer=(matched_array_filtered[:,14+4]-matched_array_filtered[:,4])>1
arrowup = u'$\u2191$'

# select object fainter than 23 mag in g-band today to represent with arrows on the plot
matched_array_filtered_and_more_than_2_mag_dimmer=matched_array_filtered[selection_of_objects_2_mag_dimmer]
matched_array_filtered_and_more_than_2_mag_dimmer_and_faint_now=matched_array_filtered_and_more_than_2_mag_dimmer[matched_array_filtered_and_more_than_2_mag_dimmer[:,8+4]>23]

matched_array_filtered_and_more_than_1_mag_dimmer=matched_array_filtered[selection_of_objects_1_mag_dimmer]
matched_array_filtered_and_more_than_1_mag_dimmer_and_faint_now=matched_array_filtered_and_more_than_1_mag_dimmer[matched_array_filtered_and_more_than_1_mag_dimmer[:,8+4]>23]


plt.figure(figsize=(20,7))
plt.subplot(131)

plt.subplots_adjust(left=-0.10)

plt.scatter(matched_array_filtered[:,4],matched_array_filtered[:,14+4],s=1)
plt.scatter(matched_array_filtered[:,4][selection_of_objects_1_mag_dimmer],
            matched_array_filtered[:,14+4][selection_of_objects_1_mag_dimmer],s=3)
plt.scatter(matched_array_filtered[:,4][selection_of_objects_2_mag_dimmer],
            matched_array_filtered[:,14+4][selection_of_objects_2_mag_dimmer],s=5,color='red')
# first plot those that are 1 mag dimmer and faint
plt.scatter(matched_array_filtered_and_more_than_1_mag_dimmer_and_faint_now[:,4],
            np.full(len(matched_array_filtered_and_more_than_1_mag_dimmer_and_faint_now),22.8),s=30,color='orange',marker=arrowup)
# and then those that are 2 mag dimmer and faint (which will overwrite correct orange objects)
plt.scatter(matched_array_filtered_and_more_than_2_mag_dimmer_and_faint_now[:,4],
            np.full(len(matched_array_filtered_and_more_than_2_mag_dimmer_and_faint_now),22.8),s=30,color='red',marker=arrowup)

plt.scatter(matched_array_filtered[mag_selection][:,4],matched_array_filtered[mag_selection][:,14+4],s=1,marker='*')
plt.scatter(matched_array_filtered[np.all((selection_of_objects_1_mag_dimmer,mag_selection),axis=0)][:,4],
            matched_array_filtered[np.all((selection_of_objects_1_mag_dimmer,mag_selection),axis=0)][:,14+4],s=50,marker='*',color='orange',label='dec<10, 10*16<ra<15*16')
plt.scatter(matched_array_filtered[np.all((selection_of_objects_2_mag_dimmer,mag_selection),axis=0)][:,4],
            matched_array_filtered[np.all((selection_of_objects_2_mag_dimmer,mag_selection),axis=0)][:,14+4],s=50,marker='*',color='red',label='dec<10, 10*16<ra<15*16')

plt.scatter(matched_array_MacLeod_SDSS[:,4],matched_array_MacLeod_SDSS[:,18],s=20,color='black')

plt.scatter([], [], c='orange', s=25,label='$\Delta g>1$')
plt.scatter([], [], c='red',  s=25,label='$\Delta g>2$')
plt.scatter([], [], c='black', s=25,label='MacLeod+ 2019, catalog')
plt.legend(loc=4, fontsize=14)

plt.xlabel('SDSS psf-g magnitude')
plt.ylabel('HSC psf-g magnitude')
plt.plot(range(0,100),range(0,100),color='black',ls='--')
plt.xlim(17,23)
plt.ylim(17,23)

plt.subplot(132)

plt.scatter(matched_array_filtered[:,6],matched_array_filtered[:,14+6],s=1)
plt.scatter(matched_array_filtered[:,6][selection_of_objects_1_mag_dimmer],matched_array_filtered[:,14+6][selection_of_objects_1_mag_dimmer],s=3,color='orange')
plt.scatter(matched_array_filtered[:,6][np.all((selection_of_objects_1_mag_dimmer,mag_selection),axis=0)],matched_array_filtered[:,14+6][np.all((selection_of_objects_1_mag_dimmer,mag_selection),axis=0)],s=50,color='orange',marker='*')
plt.scatter(matched_array_filtered[:,6][selection_of_objects_2_mag_dimmer],matched_array_filtered[:,14+6][selection_of_objects_2_mag_dimmer],s=5,color='red')
plt.scatter(matched_array_filtered[np.all((selection_of_objects_2_mag_dimmer,mag_selection),axis=0)][:,6],
            matched_array_filtered[np.all((selection_of_objects_2_mag_dimmer,mag_selection),axis=0)][:,14+6],s=50,marker='*',color='red',label='dec<10, 10*16<ra<15*16')


plt.scatter(matched_array_MacLeod_SDSS[:,6],matched_array_MacLeod_SDSS[:,20],s=20,color='black')

plt.xlabel('SDSS psf-r magnitude')
plt.ylabel('HSC psf-r magnitude')
plt.plot(range(0,100),range(0,100),color='black',ls='--')
plt.xlim(17,23)
plt.ylim(17,23)

plt.subplot(133)

plt.scatter(matched_array_filtered[:,8],matched_array_filtered[:,14+8],s=1)
plt.scatter(matched_array_filtered[:,8][selection_of_objects_1_mag_dimmer],matched_array_filtered[:,14+8][selection_of_objects_1_mag_dimmer],s=3)
plt.scatter(matched_array_filtered[:,8][np.all((selection_of_objects_1_mag_dimmer,mag_selection),axis=0)],matched_array_filtered[:,14+8][np.all((selection_of_objects_1_mag_dimmer,mag_selection),axis=0)],s=50,color='orange',marker='*')

plt.scatter(matched_array_filtered[:,8][selection_of_objects_2_mag_dimmer],matched_array_filtered[:,14+8][selection_of_objects_2_mag_dimmer],s=5,color='red')

plt.scatter(matched_array_filtered[np.all((selection_of_objects_2_mag_dimmer,mag_selection),axis=0)][:,8],
            matched_array_filtered[np.all((selection_of_objects_2_mag_dimmer,mag_selection),axis=0)][:,14+8],s=50,marker='*',color='red',label='dec<10, 10*16<ra<15*16')

plt.scatter(matched_array_MacLeod_SDSS[:,8],matched_array_MacLeod_SDSS[:,22],s=20,color='black')

plt.xlabel('SDSS psf-i magnitude')
plt.ylabel('HSC psf-i magnitude')
plt.plot(range(0,100),range(0,100),color='black',ls='--')
plt.xlim(17,23)
plt.ylim(17,23)
plt.show()

In [ ]:
ra_Mag_selected=(matched_array_filtered[(matched_array_filtered[:,1]<10)&(matched_array_filtered[:,2]<24*15)&(matched_array_filtered[:,2]>0*15)])[:,1]
dec_Mag_selected=(matched_array_filtered[(matched_array_filtered[:,1]<10)&(matched_array_filtered[:,2]<24*15)&(matched_array_filtered[:,2]>0*15)])[:2]

In [ ]:
plt.scatter(matched_array_filtered[mag_selection][:,4],matched_array_filtered[mag_selection][:,14+4],s=1,marker='*')
plt.scatter(matched_array_filtered[np.all((selection_of_objects_1_mag_dimmer,mag_selection),axis=0)][:,4],
            matched_array_filtered[np.all((selection_of_objects_1_mag_dimmer,mag_selection),axis=0)][:,14+4],s=10,marker='*')
plt.scatter(matched_array_filtered[np.all((selection_of_objects_2_mag_dimmer,mag_selection),axis=0)][:,4],
            matched_array_filtered[np.all((selection_of_objects_2_mag_dimmer,mag_selection),axis=0)][:,14+4],s=5,color='red',marker='*')
plt.show()
# first plot those that are 1 mag dimmer and faint
#plt.scatter(matched_array_filtered_and_more_than_1_mag_dimmer_and_faint_now[:,4],
#            np.full(len(matched_array_filtered_and_more_than_1_mag_dimmer_and_faint_now),22.8),s=30,color='orange',marker=arrowup)
# and then those that are 2 mag dimmer and faint (which will overwrite correct orange objects)
#plt.scatter(matched_array_filtered_and_more_than_2_mag_dimmer_and_faint_now[:,4],
#            np.full(len(matched_array_filtered_and_more_than_2_mag_dimmer_and_faint_now),22.8),s=30,color='red',marker=arrowup)

# Combining both c-model and psf magnitudes and output

In [21]:
# to run this you have to run Section 2.1. - Initial analysis (above)

# sdss psf-g band mag - HSC psf-g band mag
g_mag_dif=(matched_array_filtered[:,4]-matched_array_filtered[:,14+4]).astype(float)

# sdss psf-r band mag - HSC psf-r band mag
r_mag_dif=(matched_array_filtered[:,6]-matched_array_filtered[:,14+6]).astype(float)

# sdss psf-i band mag - HSC psf-i band
i_mag_dif=(matched_array_filtered[:,8]-matched_array_filtered[:,14+8]).astype(float)



# error sdss g band mag - HSC g band mag
g_mag_dif_err=np.sqrt(((matched_array_filtered[:,5]).astype(float))**2+((matched_array_filtered[:,14+5]).astype(float))**2)

# Error for the sdss psf-r band magnitude - HSC psf-r band magnitude
r_mag_dif_err=np.sqrt(((matched_array_filtered[:,7]).astype(float))**2+((matched_array_filtered[:,14+7]).astype(float))**2)

# Error for the sdss psf-i band magnitude - HSC psf-i band magnitude
i_mag_dif_err=np.sqrt(((matched_array_filtered[:,9]).astype(float))**2+((matched_array_filtered[:,14+9]).astype(float))**2)




# sdss psf-g band mag - HSC cmodel-g band mag
gcmodel_mag_dif=(matched_array_filtered[:,4]-matched_array_filtered[:,4+8]).astype(float)

# sdss psf-r band mag - HSC cmodel-r band mag
rcmodel_mag_dif=(matched_array_filtered[:,6]-matched_array_filtered[:,6+8]).astype(float)

# sdss psf-i band mag - HSC cmodel-i band
icmodel_mag_dif=(matched_array_filtered[:,8]-matched_array_filtered[:,8+8]).astype(float)



# error sdss g band mag - HSC cmodel-g band mag
gcmodel_mag_dif_err=np.sqrt(((matched_array_filtered[:,5]).astype(float))**2+((matched_array_filtered[:,5+8]).astype(float))**2)

# Error for the SDSS psf-r band magnitude - HSC cmodel-r band magnitude
rcmodel_mag_dif_err=np.sqrt(((matched_array_filtered[:,7]).astype(float))**2+((matched_array_filtered[:,7+8]).astype(float))**2)

# Error for the SDSS psf-i band magnitude - HSC cmodel-i band magnitude
icmodel_mag_dif_err=np.sqrt(((matched_array_filtered[:,9]).astype(float))**2+((matched_array_filtered[:,9+8]).astype(float))**2)

In [22]:
# insert differences in the catalog
matched_array_filtered_with_g_mag_dif=np.insert(matched_array_filtered, 4, g_mag_dif, axis=1)
matched_array_filtered_with_g_mag_dif_and_err=np.insert(matched_array_filtered_with_g_mag_dif, 5, g_mag_dif_err, axis=1)

matched_array_filtered_with_r_mag_dif=np.insert(matched_array_filtered, 6, r_mag_dif, axis=1)
matched_array_filtered_with_r_mag_dif_and_err=np.insert(matched_array_filtered_with_r_mag_dif, 7, r_mag_dif_err, axis=1)

matched_array_filtered_with_i_mag_dif=np.insert(matched_array_filtered, 8, i_mag_dif, axis=1)
matched_array_filtered_with_i_mag_dif_and_err=np.insert(matched_array_filtered_with_i_mag_dif, 9, i_mag_dif_err, axis=1)

# insert differences in the catalog
matched_array_filtered_with_gcmodel_mag_dif=np.insert(matched_array_filtered, 4, gcmodel_mag_dif, axis=1)
matched_array_filtered_with_gcmodel_mag_dif_and_err=np.insert(matched_array_filtered_with_gcmodel_mag_dif, 5, gcmodel_mag_dif_err, axis=1)

matched_array_filtered_with_rcmodel_mag_dif=np.insert(matched_array_filtered, 6, rcmodel_mag_dif, axis=1)
matched_array_filtered_with_rcmodel_mag_dif_and_err=np.insert(matched_array_filtered_with_rcmodel_mag_dif, 7, rcmodel_mag_dif_err, axis=1)

matched_array_filtered_with_icmodel_mag_dif=np.insert(matched_array_filtered, 8, icmodel_mag_dif, axis=1)
matched_array_filtered_with_icmodel_mag_dif_and_err=np.insert(matched_array_filtered_with_icmodel_mag_dif, 9, icmodel_mag_dif_err, axis=1)

#np.save("/home/tpena01/AGN_variability_project/matched_array_filtered_with_g_mag_dif_and_err.npy",matched_array_filtered_with_g_mag_dif_and_err)
#np.save("/home/tpena01/AGN_variability_project/matched_array_filtered_with_r_mag_dif_and_err.npy",matched_array_filtered_with_r_mag_dif_and_err)
#np.save("/home/tpena01/AGN_variability_project/matched_array_filtered_with_i_mag_dif_and_err.npy",matched_array_filtered_with_i_mag_dif_and_err)

#np.save("/home/tpena01/AGN_variability_project/matched_array_filtered_with_gcmodel_mag_dif_and_err.npy",matched_array_filtered_with_gcmodel_mag_dif_and_err)
#np.save("/home/tpena01/AGN_variability_project/matched_array_filtered_with_rcmodel_mag_dif_and_err.npy",matched_array_filtered_with_rcmodel_mag_dif_and_err)
#np.save("/home/tpena01/AGN_variability_project/matched_array_filtered_with_icmodel_mag_dif_and_err.npy",matched_array_filtered_with_icmodel_mag_dif_and_err)

In [23]:
# insert psfSDSS - psfHSC differences in the catalog
new_matched_data_array = np.insert(matched_array_filtered, 4, g_mag_dif, axis=1)
new_matched_data_array = np.insert(new_matched_data_array, 5, g_mag_dif_err, axis=1)
new_matched_data_array = np.insert(new_matched_data_array, 6, gcmodel_mag_dif, axis=1)
new_matched_data_array = np.insert(new_matched_data_array, 7, gcmodel_mag_dif_err, axis=1)
new_matched_data_array = np.insert(new_matched_data_array, 10, r_mag_dif, axis=1)
new_matched_data_array = np.insert(new_matched_data_array, 11, r_mag_dif_err, axis=1)
new_matched_data_array = np.insert(new_matched_data_array, 12, rcmodel_mag_dif, axis=1)
new_matched_data_array = np.insert(new_matched_data_array, 13, rcmodel_mag_dif_err, axis=1)
new_matched_data_array = np.insert(new_matched_data_array, 16, i_mag_dif, axis=1)
new_matched_data_array = np.insert(new_matched_data_array, 17, i_mag_dif_err, axis=1)
new_matched_data_array = np.insert(new_matched_data_array, 18, icmodel_mag_dif, axis=1)
new_matched_data_array = np.insert(new_matched_data_array, 19, icmodel_mag_dif_err, axis=1)


#np.save("/home/tpena01/AGN_variability_project/new_matched_data_array.npy", new_matched_data_array)

In [26]:
# extra step to get our g_mag data looking nice when it's exported to .txt
g_mag_data = np.zeros((len(matched_array_filtered_with_g_mag_dif_and_err)), dtype={'names':['SDSS ID','ra (SDSS)', 'dec (SDSS)', 'redshift', 'Delta g-band','Delta g-band err', 'g-band (SDSS)', 'g-band err (SDSS)',
                                                                                   'r-band (SDSS)','r-band err (SDSS)','i-band (SDSS)','i-band err (SDSS)','ra (HSC)', 'dec (HSC)',
                                                                                   'g_cmodel_mag (HSC)','g_cmodel_magsigma (HSC)','r_cmodel_mag (HSC)','r_cmodel_magsigma (HSC)',
                                                                                   'i_cmodel_mag (HSC)','i_cmodel_magsigma (HSC)','g_psfflux_mag (HSC)','g_psfflux_magsigma (HSC)',
                                                                                  'r_psfflux_mag (HSC)','r_psfflux_magsigma (HSC)','i_psfflux_mag (HSC)', 'i_psfflux_magsigma (HSC)',
                                                                                   'object_id (HSC)'],
                          'formats':['U18','f2', 'f4','f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','i8']})

for i in range(matched_array_filtered_with_g_mag_dif_and_err.shape[1]):
    g_mag_data[g_mag_data.dtype.names[i]]=matched_array_filtered_with_g_mag_dif_and_err[:,i]

# extra step to get our r_mag data looking nice when it's exported to .txt
r_mag_data = np.zeros((len(matched_array_filtered_with_r_mag_dif_and_err)), dtype={'names':['SDSS ID','ra (SDSS)', 'dec (SDSS)', 'redshift', 'g-band (SDSS)', 'g-band err (SDSS)',  'Delta r-band','Delta r-band err',
                                                                                   'r-band (SDSS)','r-band err (SDSS)','i-band (SDSS)','i-band err (SDSS)','ra (HSC)', 'dec (HSC)',
                                                                                   'g_cmodel_mag (HSC)','g_cmodel_magsigma (HSC)','r_cmodel_mag (HSC)','r_cmodel_magsigma (HSC)',
                                                                                   'i_cmodel_mag (HSC)','i_cmodel_magsigma (HSC)','g_psfflux_mag (HSC)','g_psfflux_magsigma (HSC)',
                                                                                  'r_psfflux_mag (HSC)','r_psfflux_magsigma (HSC)','i_psfflux_mag (HSC)', 'i_psfflux_magsigma (HSC)',
                                                                                   'object_id (HSC)'],
                          'formats':['U18','f2', 'f4','f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','i8']})

for i in range(matched_array_filtered_with_r_mag_dif_and_err.shape[1]):
    r_mag_data[r_mag_data.dtype.names[i]]=matched_array_filtered_with_r_mag_dif_and_err[:,i]

# extra step to get our i_mag data looking nice when it's exported to .txt
i_mag_data = np.zeros((len(matched_array_filtered_with_i_mag_dif_and_err)), dtype={'names':['SDSS ID','ra (SDSS)', 'dec (SDSS)', 'redshift', 'g-band (SDSS)', 'g-band err (SDSS)',
                                                                                   'r-band (SDSS)','r-band err (SDSS)', 'Delta i-band','Delta i-band err', 'i-band (SDSS)','i-band err (SDSS)','ra (HSC)', 'dec (HSC)',
                                                                                   'g_cmodel_mag (HSC)','g_cmodel_magsigma (HSC)','r_cmodel_mag (HSC)','r_cmodel_magsigma (HSC)',
                                                                                   'i_cmodel_mag (HSC)','i_cmodel_magsigma (HSC)','g_psfflux_mag (HSC)','g_psfflux_magsigma (HSC)',
                                                                                  'r_psfflux_mag (HSC)','r_psfflux_magsigma (HSC)','i_psfflux_mag (HSC)', 'i_psfflux_magsigma (HSC)',
                                                                                   'object_id (HSC)'],
                          'formats':['U18','f2', 'f4','f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','i8']})

for i in range(matched_array_filtered_with_i_mag_dif_and_err.shape[1]):
    i_mag_data[i_mag_data.dtype.names[i]]=matched_array_filtered_with_i_mag_dif_and_err[:,i]

In [27]:
# Same as above. Only difference is that this time, we're doing everything wtih cmodel data, rather than psf data.
gcmodel_mag_data = np.zeros((len(matched_array_filtered_with_gcmodel_mag_dif_and_err)), dtype={'names':['SDSS ID','ra (SDSS)', 'dec (SDSS)', 'redshift', 'Delta g-band','Delta g-band err', 'g-band (SDSS)', 'g-band err (SDSS)',
                                                                                   'r-band (SDSS)','r-band err (SDSS)','i-band (SDSS)','i-band err (SDSS)','ra (HSC)', 'dec (HSC)',
                                                                                   'g_cmodel_mag (HSC)','g_cmodel_magsigma (HSC)','r_cmodel_mag (HSC)','r_cmodel_magsigma (HSC)',
                                                                                   'i_cmodel_mag (HSC)','i_cmodel_magsigma (HSC)','g_psfflux_mag (HSC)','g_psfflux_magsigma (HSC)',
                                                                                  'r_psfflux_mag (HSC)','r_psfflux_magsigma (HSC)','i_psfflux_mag (HSC)', 'i_psfflux_magsigma (HSC)',
                                                                                   'object_id (HSC)'],
                          'formats':['U18','f2', 'f4','f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','i8']})

for i in range(matched_array_filtered_with_gcmodel_mag_dif_and_err.shape[1]):
    gcmodel_mag_data[gcmodel_mag_data.dtype.names[i]]=matched_array_filtered_with_gcmodel_mag_dif_and_err[:,i]

rcmodel_mag_data = np.zeros((len(matched_array_filtered_with_rcmodel_mag_dif_and_err)), dtype={'names':['SDSS ID','ra (SDSS)', 'dec (SDSS)', 'redshift', 'g-band (SDSS)', 'g-band err (SDSS)',  'Delta r-band','Delta r-band err',
                                                                                   'r-band (SDSS)','r-band err (SDSS)','i-band (SDSS)','i-band err (SDSS)','ra (HSC)', 'dec (HSC)',
                                                                                   'g_cmodel_mag (HSC)','g_cmodel_magsigma (HSC)','r_cmodel_mag (HSC)','r_cmodel_magsigma (HSC)',
                                                                                   'i_cmodel_mag (HSC)','i_cmodel_magsigma (HSC)','g_psfflux_mag (HSC)','g_psfflux_magsigma (HSC)',
                                                                                  'r_psfflux_mag (HSC)','r_psfflux_magsigma (HSC)','i_psfflux_mag (HSC)', 'i_psfflux_magsigma (HSC)',
                                                                                   'object_id (HSC)'],
                          'formats':['U18','f2', 'f4','f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','i8']})

for i in range(matched_array_filtered_with_rcmodel_mag_dif_and_err.shape[1]):
    rcmodel_mag_data[rcmodel_mag_data.dtype.names[i]]=matched_array_filtered_with_rcmodel_mag_dif_and_err[:,i]

icmodel_mag_data = np.zeros((len(matched_array_filtered_with_icmodel_mag_dif_and_err)), dtype={'names':['SDSS ID','ra (SDSS)', 'dec (SDSS)', 'redshift', 'g-band (SDSS)', 'g-band err (SDSS)',
                                                                                   'r-band (SDSS)','r-band err (SDSS)', 'Delta i-band','Delta i-band err', 'i-band (SDSS)','i-band err (SDSS)','ra (HSC)', 'dec (HSC)',
                                                                                   'g_cmodel_mag (HSC)','g_cmodel_magsigma (HSC)','r_cmodel_mag (HSC)','r_cmodel_magsigma (HSC)',
                                                                                   'i_cmodel_mag (HSC)','i_cmodel_magsigma (HSC)','g_psfflux_mag (HSC)','g_psfflux_magsigma (HSC)',
                                                                                  'r_psfflux_mag (HSC)','r_psfflux_magsigma (HSC)','i_psfflux_mag (HSC)', 'i_psfflux_magsigma (HSC)',
                                                                                   'object_id (HSC)'],
                          'formats':['U18','f2', 'f4','f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','i8']})

for i in range(matched_array_filtered_with_icmodel_mag_dif_and_err.shape[1]):
    icmodel_mag_data[icmodel_mag_data.dtype.names[i]]=matched_array_filtered_with_icmodel_mag_dif_and_err[:,i]

In [28]:
# extra step to get our g_mag data looking nice when it's exported to .txt
catalog_with_changes = np.zeros((len(new_matched_data_array)), dtype={'names':['SDSS ID','ra (SDSS)', 'dec (SDSS)', 'redshift', 'Delta g-band (psf SDSS - psf HSC)','Delta g-band err (psf SDSS - psf HSC)',
                                                                                  'Delta g-band (psf SDSS - cmodel HSC)', 'Delta g-band err (psf SDSS - cmodel HSC)', 'g-band (SDSS)', 'g-band err (SDSS)',
                                                                                   'Delta r-band (psf SDSS - psf HSC)','Delta r-band err (psf SDSS - psf HSC)', 'Delta r-band (psf SDSS - cmodel HSC)','Delta r-band err (psf SDSS - cmodel HSC)', 'r-band (SDSS)','r-band err (SDSS)',
                                                                                   'Delta i-band (psf SDSS - psf HSC)','Delta i-band err (psf SDSS - psf HSC)', 'Delta i-band (psf SDSS - cmodel HSC)','Delta i-band err (psf SDSS - cmodel HSC)', 'i-band (SDSS)','i-band err (SDSS)',
                                                                                   'ra (HSC)', 'dec (HSC)', 'g_cmodel_mag (HSC)','g_cmodel_magsigma (HSC)','r_cmodel_mag (HSC)','r_cmodel_magsigma (HSC)',
                                                                                   'i_cmodel_mag (HSC)','i_cmodel_magsigma (HSC)','g_psfflux_mag (HSC)','g_psfflux_magsigma (HSC)',
                                                                                  'r_psfflux_mag (HSC)','r_psfflux_magsigma (HSC)','i_psfflux_mag (HSC)', 'i_psfflux_magsigma (HSC)',
                                                                                   'object_id (HSC)'],
                          'formats':['U18','f2', 'f4','f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','f4', 'f4', 'f4','i8']})

for i in range(new_matched_data_array.shape[1]):
    catalog_with_changes[catalog_with_changes.dtype.names[i]] = new_matched_data_array[:,i]


In [ ]:
#np.save("/home/tpena01/AGN_variability_project/catalog_with_changes", catalog_with_changes)

In [ ]:
#g_mag_data = np.load("/home/tpena01/AGN_variability_project/catalog_with_changes")

In [ ]:
#np.savetxt("/home/tpena01/AGN_variability_project/catalog_with_changes.csv", catalog_with_changes, delimiter=",",
#           header="#List of SDSS AGN from DR7 that are found in HSC survey, with their magnitudes \n#created on Feb 16, 2019\n#code at https://github.com/nevencaplar/Variability/tree/master/HSC \n#@Neven Caplar, Princeton University \n#ncaplar@princeton.edu\n# \n#SDSS ID \n#ra (SDSS) \n#dec (SDSS) \n#redshift \n#Delta g-band (g-band psf SDSS magnitude - g_psfflux_mag (HSC))  \n#Delta g-band err (sqrt(g-band psf err (SDSS)**2+g_psfflux_magsigma (HSC)**2)) \n#g-band (SDSS - all SDSS magnitudes are psf magnitudes) \n#g-band err (SDSS)  \n#r-band (SDSS)  \n#r-band err (SDSS) \n#i-band (SDSS) \n#i-band err (SDSS)  \n#ra (HSC)  \n#dec (HSC)  \n#g_cmodel_mag (HSC)  \n#g_cmodel_magsigma (HSC)  \n#r_cmodel_mag (HSC)  \n#r_cmodel_magsigma (HSC)  \n#i_cmodel_mag (HSC) \n# i_cmodel_magsigma (HSC)  \n#g_psfflux_mag (HSC)  \n# g_psfflux_magsigma (HSC) \n#r_psfflux_mag (HSC) \n#r_psfflux_magsigma (HSC) \n#i_psfflux_mag (HSC) \n#i_psfflux_magsigma (HSC) \n#object_id (HSC) \n  ", comments='',fmt='%s')

# Creating selection for Spectro follow up

In [ ]:
data_selected=g_mag_data[np.all((selection_of_objects_actual_1_mag_dimmer,mag_selection),axis=0)]
data_selected_cleaned=np.delete(data_selected,11)

In [ ]:
SDSS_name_MacLeod=[]
for i in range(len(matched_array_MacLeod_SDSS)):
    SDSS_name_MacLeod.append(matched_array_MacLeod_SDSS[i][0])

In [ ]:
SDSS_name_MacLeod

In [ ]:
data_selected_cleaned_priority_1=data_selected_cleaned[data_selected_cleaned['Delta g-band']<-1.25]
data_selected_cleaned_priority_1

In [ ]:
data_selected_cleaned_priority_2=data_selected_cleaned[data_selected_cleaned['Delta g-band']>-1.25]

In [ ]:
np.savetxt("/home/tpena01/AGN_variability_project/data_selected_cleaned_priority_1.csv", data_selected_cleaned_priority_1, delimiter=",", 
           header="#List of SDSS AGN from DR7 that are found in HSC survey, with their magnitudes \n#Difference between HSC and SDSS larger than 1 mag \n#Within Magellan footprint \n#created on Mar 22, 2019\n#code at https://github.com/nevencaplar/Variability/tree/master/HSC \n#@Neven Caplar, Princeton University \n#ncaplar@princeton.edu\n# \n#SDSS ID \n#dec (SDSS) \n#ra (SDSS) \n#redshift \n#Delta g-band (g-band psf SDSS magnitude - g_psfflux_mag (HSC))  \n#Delta g-band err (sqrt(g-band psf err (SDSS)**2+g_psfflux_magsigma (HSC)**2)) \n#g-band (SDSS - all SDSS magnitudes are psf magnitudes) \n#g-band err (SDSS)  \n#r-band (SDSS)  \n#r-band err (SDSS) \n#i-band (SDSS) \n#i-band err (SDSS)  \n#ra (HSC)  \n#dec (HSC)  \n#g_cmodel_mag (HSC)  \n#g_cmodel_magsigma (HSC)  \n#r_cmodel_mag (HSC)  \n#r_cmodel_magsigma (HSC)  \n#i_cmodel_mag (HSC) \n# i_cmodel_magsigma (HSC)  \n#g_psfflux_mag (HSC)  \n# g_psfflux_magsigma (HSC) \n#r_psfflux_mag (HSC) \n#r_psfflux_magsigma (HSC) \n#i_psfflux_mag (HSC) \n#i_psfflux_magsigma (HSC) \n#object_id (HSC) \n  ", comments='',fmt='%s')

In [ ]:
np.savetxt("/home/tpena01/AGN_variability_project/data_selected_cleaned_priority_2.csv", data_selected_cleaned_priority_2, delimiter=",", 
           header="#List of SDSS AGN from DR7 that are found in HSC survey, with their magnitudes \n#Difference between HSC and SDSS larger than 1 mag \n#Within Magellan footprint \n#created on Mar 22, 2019\n#code at https://github.com/nevencaplar/Variability/tree/master/HSC \n#@Neven Caplar, Princeton University \n#ncaplar@princeton.edu\n# \n#SDSS ID \n#dec (SDSS) \n#ra (SDSS) \n#redshift \n#Delta g-band (g-band psf SDSS magnitude - g_psfflux_mag (HSC))  \n#Delta g-band err (sqrt(g-band psf err (SDSS)**2+g_psfflux_magsigma (HSC)**2)) \n#g-band (SDSS - all SDSS magnitudes are psf magnitudes) \n#g-band err (SDSS)  \n#r-band (SDSS)  \n#r-band err (SDSS) \n#i-band (SDSS) \n#i-band err (SDSS)  \n#ra (HSC)  \n#dec (HSC)  \n#g_cmodel_mag (HSC)  \n#g_cmodel_magsigma (HSC)  \n#r_cmodel_mag (HSC)  \n#r_cmodel_magsigma (HSC)  \n#i_cmodel_mag (HSC) \n# i_cmodel_magsigma (HSC)  \n#g_psfflux_mag (HSC)  \n# g_psfflux_magsigma (HSC) \n#r_psfflux_mag (HSC) \n#r_psfflux_magsigma (HSC) \n#i_psfflux_mag (HSC) \n#i_psfflux_magsigma (HSC) \n#object_id (HSC) \n  ", comments='',fmt='%s')

# Small investigation of the properties of sources selected as variable

## Graphing function

In [29]:
def make_graph(data_array, data_col=4, obj_per_bin=100, sortby='redshift', smoothing_method='mean', num_to_choose=None, ylim=None):
    '''
    Graphing function. 
    Input is new_matched_data_array and your favorite set of parameters.
    Output is a graph.
    '''
    # Set variables
    redshift_col = 3
    error_col = data_col + 1
    g_mag_SDSS_col = 8
    
    # Failsafe in case the user forgets to select a number of objects per bin.
    if (num_to_choose == None):
        num_to_choose = obj_per_bin
        
    # Extract the data that we're going to use.
    delta_band_and_redshift = data_array[:, [data_col, redshift_col, error_col]]
    
    
    if (sortby == 'delta_band'):
        # Split our data into bins. Each bin has "obj_per_bin" elements
        delta_band_and_redshift_sorted_and_split=np.array_split(delta_band_and_redshift[np.argsort(delta_band_and_redshift[:,0])], int(len(delta_band_and_redshift)/obj_per_bin))
        
        # The following pair of for loops sorts and smooths the data.
        res_delta_band_and_redshift = []
        res_delta_band_and_redshift_median = []
        res_delta_std = []
        
        for i in range(len(delta_band_and_redshift_sorted_and_split)):
            res_delta_band_and_redshift.append(np.mean(delta_band_and_redshift_sorted_and_split[i],axis=0))
            res_delta_band_and_redshift_median.append(np.median(delta_band_and_redshift_sorted_and_split[i],axis=0))
            res_delta_std.append(np.array(delta_band_and_redshift_sorted_and_split[i][:,:], dtype=np.float64).std(axis=0))

        # Now that our data is in a graph
        res_delta_band_and_redshift=np.array(res_delta_band_and_redshift)
        res_delta_band_and_redshift_median=np.array(res_delta_band_and_redshift_median)
        res_delta_std=np.array(res_delta_std)

        
        if (smoothing_method == 'mean'):
            # Plotting
            matplotlib.rcParams.update({'font.size': 12})
            plt.figure(figsize=(18,6))
            plt.title('negative $\Delta$ BAND-band values= fainter in HSC; positive $\Delta$ BAND-band values= brighter in HSC; DATA ORIGIN UNKNOWN')
            plt.errorbar(res_delta_band_and_redshift[:,0], res_delta_band_and_redshift[:,1], yerr=res_delta_std[:,1])   
            plt.xlabel('$\Delta$ BAND-band (SDSS - HSC)')
            plt.ylabel('redshift')
            if (ylim != None):
                plt.ylim(ylim[0], ylim[1])
            else:
                pass
            plt.show()
        
        elif (smoothing_method == 'median'):
            # Plotting
            matplotlib.rcParams.update({'font.size': 12})
            plt.figure(figsize=(18,6))
            plt.title('negative $\Delta$ BAND-band values= fainter in HSC; positive $\Delta$ BAND-band values= brighter in HSC; DATA ORIGIN UNKNOWN')
            plt.errorbar(res_delta_band_and_redshift_median[:,0], res_delta_band_and_redshift_median[:,1], yerr=res_delta_std[:,1])
            plt.xlabel('$\Delta$ g-band (SDSS - HSC)')
            plt.ylabel('redshift')
            if (ylim != None):
                plt.ylim(ylim[0], ylim[1])
            else:
                pass
            plt.show()
            
        else:
            # Throw an error here later
            pass
    
    
    elif (sortby == 'redshift'):
        delta_band_and_redshift_sorted_and_split=np.array_split(delta_band_and_redshift[np.argsort(delta_band_and_redshift[:,1])], int(len(delta_band_and_redshift)/obj_per_bin))

        # Smoothing
        res_delta_band_and_redshift = []
        res_delta_band_and_redshift_median = []
        res_delta_std = []
        

        for i in range(len(delta_band_and_redshift_sorted_and_split)):
            res_delta_band_and_redshift.append(np.mean(delta_band_and_redshift_sorted_and_split[i],axis=0))
            res_delta_band_and_redshift_median.append(np.median(delta_band_and_redshift_sorted_and_split[i],axis=0))
            res_delta_std.append(np.array(delta_band_and_redshift_sorted_and_split[i][:,:], dtype=np.float64).std(axis=0))
                
        res_delta_band_and_redshift=np.array(res_delta_band_and_redshift)
        res_delta_band_and_redshift_median=np.array(res_delta_band_and_redshift_median)
        res_delta_std=np.array(res_delta_std)

                
        second_deg_fit=np.poly1d(np.polyfit(res_delta_band_and_redshift[:,1].astype(float), res_delta_band_and_redshift[:,0].astype(float), 2))
    
        
        if (smoothing_method == 'mean'):
            # PLotting
            matplotlib.rcParams.update({'font.size': 12})
            plt.figure(figsize=(18,6))
            plt.title('negative $\Delta$ BAND-band values= fainter in HSC; positive $\Delta$ BAND-band values= brighter in HSC; DATA ORIGIN UNKNOWN')
            plt.errorbar(res_delta_band_and_redshift[:,1], res_delta_band_and_redshift[:,0], yerr=res_delta_std[:,0])
            plt.plot(res_delta_band_and_redshift[:,1], second_deg_fit(res_delta_band_and_redshift[:,1].astype(float)), ls='--', color='orange')    
            plt.ylabel('$\Delta$ BAND-band (SDSS - HSC)')
            plt.xlabel('redshift')
            if (ylim != None):
                plt.ylim(ylim[0], ylim[1])
            else:
                pass
            plt.show()
        
        elif (smoothing_method == 'median'):
            # PLotting
            matplotlib.rcParams.update({'font.size': 12})
            plt.figure(figsize=(18,6))
            plt.title('negative $\Delta$ BAND-band values= fainter in HSC; positive $\Delta$ BAND-band values= brighter in HSC; DATA ORIGIN UNKNOWN')
            plt.errorbar(res_delta_band_and_redshift_median[:,1], res_delta_band_and_redshift_median[:,0], yerr=res_delta_std[:,0])
            plt.plot(res_delta_band_and_redshift_median[:,1], second_deg_fit(res_delta_band_and_redshift_median[:,1].astype(float)), ls='--', color='orange')    
            plt.ylabel('$\Delta$ BAND-band (SDSS - HSC)')
            plt.xlabel('redshift')
            if (ylim != None):
                plt.ylim(ylim[0], ylim[1])
            else:
                pass
            plt.show()
    

    elif (sortby == 'g_mag_SDSS'):

        # Add on a row that represents the g_mag in SDSS. We'll sort by this later
        delta_band_and_redshift = np.hstack((delta_band_and_redshift, np.reshape(data_array[:, g_mag_SDSS_col], (len(data_array[:]),1)) ))
        
        
        delta_band_and_redshift_sorted_and_split=np.array_split(delta_band_and_redshift[np.argsort(delta_band_and_redshift[:,1])], int(len(delta_band_and_redshift)/obj_per_bin))
        
        
        
        # Next we sort each bin by g_mag
        for i in range(len(delta_band_and_redshift_sorted_and_split)):
            delta_band_and_redshift_sorted_and_split[i] = delta_band_and_redshift_sorted_and_split[i][np.argsort(delta_band_and_redshift_sorted_and_split[i][:, delta_band_and_redshift_sorted_and_split[i].shape[1]-1])]

            

        res_delta_band_and_redshift = []
        res_delta_band_and_redshift_median = []
        res_delta_std = []
        
        if (num_to_choose < 0):
            for i in range(len(delta_band_and_redshift_sorted_and_split)):
                res_delta_band_and_redshift.append(np.mean(delta_band_and_redshift_sorted_and_split[i][num_to_choose:],axis=0))
                res_delta_std.append(np.array(delta_band_and_redshift_sorted_and_split[i][num_to_choose:][:,:], dtype=np.float64).std(axis=0))
                res_delta_band_and_redshift_median.append(np.median(delta_band_and_redshift_sorted_and_split[i][num_to_choose:],axis=0))

        elif (num_to_choose >= 0):
            for i in range(len(delta_band_and_redshift_sorted_and_split)):
                res_delta_band_and_redshift.append(np.mean(delta_band_and_redshift_sorted_and_split[i][:num_to_choose],axis=0))
                res_delta_band_and_redshift_median.append(np.median(delta_band_and_redshift_sorted_and_split[i][:num_to_choose],axis=0))
                res_delta_std.append(np.array(delta_band_and_redshift_sorted_and_split[i][:num_to_choose][:,:], dtype=np.float64).std(axis=0))

        else:
            # You guessed it, throw an error
            pass
        
        res_delta_band_and_redshift=np.array(res_delta_band_and_redshift)
        res_delta_band_and_redshift_median=np.array(res_delta_band_and_redshift_median)
        res_delta_std=np.array(res_delta_std)
        

                
        second_deg_fit=np.poly1d(np.polyfit(res_delta_band_and_redshift[:,1].astype(float), res_delta_band_and_redshift[:,0].astype(float), 2))
    
        
        if (smoothing_method == 'mean'):
            # PLotting
            matplotlib.rcParams.update({'font.size': 12})
            plt.figure(figsize=(18,6))
            plt.title('negative $\Delta$ BAND-band values= fainter in HSC; positive $\Delta$ BAND-band values= brighter in HSC; DATA ORIGIN UNKNOWN')
            plt.errorbar(res_delta_band_and_redshift[:,1], res_delta_band_and_redshift[:,0], yerr=res_delta_std[:,0])
            plt.plot(res_delta_band_and_redshift[:,1], second_deg_fit(res_delta_band_and_redshift[:,1].astype(float)), ls='--', color='orange')    
            plt.ylabel('$\Delta$ BAND-band (SDSS - HSC)')
            plt.xlabel('redshift')
            if (ylim != None):
                plt.ylim(ylim[0], ylim[1])
            else:
                pass
                
            plt.show()
        
        elif (smoothing_method == 'median'):
            # PLotting
            matplotlib.rcParams.update({'font.size': 12})
            plt.figure(figsize=(18,6))
            plt.title('negative $\Delta$ BAND-band values= fainter in HSC; positive $\Delta$ BAND-band values= brighter in HSC; DATA ORIGIN UNKNOWN')
            plt.errorbar(res_delta_band_and_redshift_median[:,1], res_delta_band_and_redshift_median[:,0], yerr=res_delta_std[:,0])
            plt.plot(res_delta_band_and_redshift_median[:,1], second_deg_fit(res_delta_band_and_redshift_median[:,1].astype(float)), ls='--', color='orange')    
            plt.ylabel('$\Delta$ BAND-band (SDSS - HSC)')
            plt.xlabel('redshift')
            if (ylim != None):
                plt.ylim(ylim[0], ylim[1])
                plt.xlim(0.1, 0)
            else:
                pass
            plt.show()
            

    elif (sortby == 'g_mag_HSC'):
        
        delta_band_and_redshift = np.hstack((delta_band_and_redshift, np.reshape(data_array[:,20], (len(data_array[:]),1)) )) 
        delta_band_and_redshift_sorted_and_split=np.array_split(delta_band_and_redshift[np.argsort(delta_band_and_redshift[:,1])], int(len(delta_band_and_redshift)/obj_per_bin))
        
        
        
        # Next we sort each bin by g_mag
        for i in range(len(delta_band_and_redshift_sorted_and_split)):
            delta_band_and_redshift_sorted_and_split[i] = delta_band_and_redshift_sorted_and_split[i][np.argsort(delta_band_and_redshift_sorted_and_split[i][:, delta_band_and_redshift_sorted_and_split[i].shape[1]-1])]

        res_delta_band_and_redshift = []
        res_delta_band_and_redshift_median = []
        res_delta_std = []
        
        if (num_to_choose <= 0):
            for i in range(len(delta_band_and_redshift_sorted_and_split)):
                res_delta_band_and_redshift.append(np.mean(delta_band_and_redshift_sorted_and_split[i][num_to_choose:],axis=0))
                res_delta_band_and_redshift_median.append(np.median(delta_band_and_redshift_sorted_and_split[i][num_to_choose:],axis=0))
                res_delta_std.append(np.array(delta_band_and_redshift_sorted_and_split[i][num_to_choose:][:,:], dtype=np.float64).std(axis=0))

        elif (num_to_choose >= 0):
            for i in range(len(delta_band_and_redshift_sorted_and_split)):
                res_delta_band_and_redshift.append(np.mean(delta_band_and_redshift_sorted_and_split[i][:num_to_choose],axis=0))
                res_delta_band_and_redshift_median.append(np.median(delta_band_and_redshift_sorted_and_split[i][:num_to_choose],axis=0))
                res_delta_std.append(np.array(delta_band_and_redshift_sorted_and_split[i][:num_to_choose][:,:], dtype=np.float64).std(axis=0))

        else:
            # You guessed it, throw an error
            pass
        
        res_delta_band_and_redshift=np.array(res_delta_band_and_redshift)
        res_delta_band_and_redshift_median=np.array(res_delta_band_and_redshift_median)
        res_delta_std=np.array(res_delta_std)
        

                
        second_deg_fit=np.poly1d(np.polyfit(res_delta_band_and_redshift[:,1].astype(float), res_delta_band_and_redshift[:,0].astype(float), 2))
    
        
        if (smoothing_method == 'mean'):
            # PLotting
            matplotlib.rcParams.update({'font.size': 12})
            plt.figure(figsize=(18,6))
            plt.title('negative $\Delta$ BAND-band values= fainter in HSC; positive $\Delta$ BAND-band values= brighter in HSC; DATA ORIGIN UNKNOWN')
            plt.errorbar(res_delta_band_and_redshift[:,1], res_delta_band_and_redshift[:,0], yerr=res_delta_std[:,0])
            plt.plot(res_delta_band_and_redshift[:,1], second_deg_fit(res_delta_band_and_redshift[:,1].astype(float)), ls='--', color='orange')    
            plt.ylabel('$\Delta$ BAND-band (SDSS - HSC)')
            plt.xlabel('redshift')
            if (ylim != None):
                plt.ylim(ylim[0], ylim[1])
                plt.xlim(0.1, 0.35)
            else:
                pass
            plt.show()
        
        elif (smoothing_method == 'median'):
            # PLotting
            matplotlib.rcParams.update({'font.size': 12})
            plt.figure(figsize=(18,6))
            plt.title('negative $\Delta$ BAND-band values= fainter in HSC; positive $\Delta$ BAND-band values= brighter in HSC; DATA ORIGIN UNKNOWN')
            plt.errorbar(res_delta_band_and_redshift_median[:,1], res_delta_band_and_redshift_median[:,0], yerr=res_delta_std[:,0])
            plt.plot(res_delta_band_and_redshift_median[:,1], second_deg_fit(res_delta_band_and_redshift_median[:,1].astype(float)), ls='--', color='orange')    
            plt.ylabel('$\Delta$ BAND-band (SDSS - HSC)')
            plt.xlabel('redshift')
            if (ylim != None):
                plt.ylim(ylim[0], ylim[1])
                plt.xlim(0.1, 0.35)
            else:
                pass
            plt.show()
    
    
    else:
        "Throw an error here later"
        pass

    return  

In [30]:
"""
DRIVER CELL
===============
data_col needs to be updated for the function to work properly!!
delta_g = 4
delta_g_cmodel=6
delta_r = 10
delta_r_cmodel = 12
delta_ i = 16
delta_i_cmodel = 18


PARAMETERS
---------------
data_array. Where you want to pull information from. 
data_col. Which column the data is in.
obj_per_bin. Self explaintory.
sortby. Options are 'redshift', 'delta_band', 'g_mag_HSC', and 'g_mag_SDSS' 
smoothing_method. Options are 'mean' and 'median'
num_to_choose=20. How many objects you want to choose to use from each bin. Only relevant when sorting due to g_band mag. Use a positive number to choose dim objects, negative to choose bright objects.
"""

make_graph(new_matched_data_array, data_col=10, obj_per_bin = 100, sortby='g_mag_SDSS', smoothing_method='median', num_to_choose=-20, ylim=(-0.4, 0.2))
make_graph(new_matched_data_array, data_col=10, obj_per_bin = 100, sortby='g_mag_SDSS', smoothing_method='mean', num_to_choose=20, ylim=(-0.4, 0.2))

## Further plots

In [ ]:
np.std(np.array(x[9][:,0]))
graphable_data = []
for i in range(len(x[9])):
    graphable_data.append(x[9][i,0])

In [ ]:
plt.hist(graphable_data)
plt.axvline(np.mean(graphable_data), color='red')
plt.hist(graphable_data[-20:])
plt.axvline(np.mean(graphable_data[-20:]), color='cyan')
plt.hist(graphable_data[:20])
plt.axvline(np.mean(graphable_data[:20]), color='black')
plt.show()

## Custom order plot

In [38]:
def DCSP(data_col=4, smoothing_method='mean', ylim=None, num_to_choose=None):
    '''
    Graphing function. 
    Input is new_matched_data_array and your favorite set of parameters.
    Output is a graph.
    '''
    # Set variables
    redshift_col = 3
    error_col = data_col + 1
    g_mag_SDSS_col = 8
    obj_per_bin = 100
    data_array = new_matched_data_array
    
    # Failsafe in case the user forgets to select a number of objects per bin.
    
    # Extract the data that we're going to use.
    delta_band_and_redshift = data_array[:, [data_col, redshift_col]]
    
    # Add on a row that represents the g_mag in SDSS. We'll sort by this later
    delta_band_and_redshift = np.hstack((delta_band_and_redshift, np.reshape(data_array[:, g_mag_SDSS_col], (len(data_array[:]),1)) ))


    delta_band_and_redshift_sorted_and_split=np.array_split(delta_band_and_redshift[np.argsort(delta_band_and_redshift[:,1])], int(len(delta_band_and_redshift)/obj_per_bin))



    # Next we sort each bin by g_mag
    for i in range(len(delta_band_and_redshift_sorted_and_split)):
        delta_band_and_redshift_sorted_and_split[i] = delta_band_and_redshift_sorted_and_split[i][np.argsort(delta_band_and_redshift_sorted_and_split[i][:, delta_band_and_redshift_sorted_and_split[i].shape[1]-1])]

    
    
    res_delta_band_and_redshift_0_20 = []
    res_delta_band_and_redshift_20_40 = []
    res_delta_band_and_redshift_40_60 = []
    res_delta_band_and_redshift_60_80 = []
    res_delta_band_and_redshift_80_100 = []
    res_delta_band_and_redshift_all = []

    res_delta_band_and_redshift_0_20_med = []
    res_delta_band_and_redshift_20_40_med = []
    res_delta_band_and_redshift_40_60_med = []
    res_delta_band_and_redshift_60_80_med = []
    res_delta_band_and_redshift_80_100_med = []
    res_delta_band_and_redshift_all_med = []
    
    res_delta_std_all = []
    
    for i in range(len(delta_band_and_redshift_sorted_and_split)):
        res_delta_band_and_redshift_0_20.append(np.mean(delta_band_and_redshift_sorted_and_split[i][:20],axis=0))
        res_delta_band_and_redshift_20_40.append(np.mean(delta_band_and_redshift_sorted_and_split[i][20:40],axis=0))
        res_delta_band_and_redshift_40_60.append(np.mean(delta_band_and_redshift_sorted_and_split[i][40:60],axis=0))
        res_delta_band_and_redshift_60_80.append(np.mean(delta_band_and_redshift_sorted_and_split[i][60:80],axis=0))
        res_delta_band_and_redshift_80_100.append(np.mean(delta_band_and_redshift_sorted_and_split[i][80:100],axis=0))
        res_delta_band_and_redshift_all.append(np.mean(delta_band_and_redshift_sorted_and_split[i],axis=0))
        
        res_delta_band_and_redshift_0_20_med.append(np.median(delta_band_and_redshift_sorted_and_split[i][:20],axis=0))
        res_delta_band_and_redshift_20_40_med.append(np.median(delta_band_and_redshift_sorted_and_split[i][20:40],axis=0))
        res_delta_band_and_redshift_40_60_med.append(np.median(delta_band_and_redshift_sorted_and_split[i][40:60],axis=0))
        res_delta_band_and_redshift_60_80_med.append(np.median(delta_band_and_redshift_sorted_and_split[i][60:80],axis=0))
        res_delta_band_and_redshift_80_100_med.append(np.median(delta_band_and_redshift_sorted_and_split[i][80:100],axis=0))
        res_delta_band_and_redshift_all_med.append(np.median(delta_band_and_redshift_sorted_and_split[i],axis=0))

        res_delta_std_all.append(np.array(delta_band_and_redshift_sorted_and_split[i][:,:], dtype=np.float64).std(axis=0))
        
    else:
        # You guessed it, throw an error
        pass

    res_delta_band_and_redshift_0_20 = np.array(res_delta_band_and_redshift_0_20)
    res_delta_band_and_redshift_20_40 = np.array(res_delta_band_and_redshift_20_40)
    res_delta_band_and_redshift_40_60 = np.array(res_delta_band_and_redshift_40_60)
    res_delta_band_and_redshift_60_80 = np.array(res_delta_band_and_redshift_60_80)
    res_delta_band_and_redshift_80_100 = np.array(res_delta_band_and_redshift_80_100)
    res_delta_band_and_redshift_all = np.array(res_delta_band_and_redshift_all)

    res_delta_band_and_redshift_0_20_med = np.array(res_delta_band_and_redshift_0_20_med)
    res_delta_band_and_redshift_20_40_med = np.array(res_delta_band_and_redshift_20_40_med)
    res_delta_band_and_redshift_40_60_med = np.array(res_delta_band_and_redshift_40_60_med)
    res_delta_band_and_redshift_60_80_med = np.array(res_delta_band_and_redshift_60_80_med)
    res_delta_band_and_redshift_80_100_med = np.array(res_delta_band_and_redshift_80_100_med)
    res_delta_band_and_redshift_all_med = np.array(res_delta_band_and_redshift_all_med)
    
    res_delta_std_all = np.array(res_delta_std_all)

    second_deg_fit_0_20 = np.poly1d(np.polyfit(res_delta_band_and_redshift_0_20[:,1].astype(float), res_delta_band_and_redshift_0_20[:, 0].astype(float), 2))
    second_deg_fit_20_40 = np.poly1d(np.polyfit(res_delta_band_and_redshift_20_40[:,1].astype(float), res_delta_band_and_redshift_20_40[:, 0].astype(float), 2))
    second_deg_fit_40_60 = np.poly1d(np.polyfit(res_delta_band_and_redshift_40_60[:,1].astype(float), res_delta_band_and_redshift_40_60[:, 0].astype(float), 2))
    second_deg_fit_60_80 = np.poly1d(np.polyfit(res_delta_band_and_redshift_60_80[:,1].astype(float), res_delta_band_and_redshift_60_80[:, 0].astype(float), 2))
    second_deg_fit_80_100 = np.poly1d(np.polyfit(res_delta_band_and_redshift_80_100[:,1].astype(float), res_delta_band_and_redshift_80_100[:, 0].astype(float), 2))
    
    second_deg_fit_0_20_med = np.poly1d(np.polyfit(res_delta_band_and_redshift_0_20_med[:,1].astype(float), res_delta_band_and_redshift_0_20_med[:, 0].astype(float), 2))
    second_deg_fit_20_40_med = np.poly1d(np.polyfit(res_delta_band_and_redshift_20_40_med[:,1].astype(float), res_delta_band_and_redshift_20_40_med[:, 0].astype(float), 2))
    second_deg_fit_40_60_med = np.poly1d(np.polyfit(res_delta_band_and_redshift_40_60_med[:,1].astype(float), res_delta_band_and_redshift_40_60_med[:, 0].astype(float), 2))
    second_deg_fit_60_80_med = np.poly1d(np.polyfit(res_delta_band_and_redshift_60_80_med[:,1].astype(float), res_delta_band_and_redshift_60_80_med[:, 0].astype(float), 2))
    second_deg_fit_80_100_med = np.poly1d(np.polyfit(res_delta_band_and_redshift_80_100_med[:,1].astype(float), res_delta_band_and_redshift_80_100_med[:, 0].astype(float), 2))


    if (smoothing_method == 'mean'):
        # PLotting
        matplotlib.rcParams.update({'font.size': 12})
        plt.figure(figsize=(18,6))
        plt.title('negative $\Delta$ BAND-band values= fainter in HSC; positive $\Delta$ BAND-band values= brighter in HSC')
        plt.errorbar(res_delta_band_and_redshift_all[:,1], res_delta_band_and_redshift_all[:,0], yerr=res_delta_std_all[:,0], color='grey', alpha=.5)
        plt.plot(res_delta_band_and_redshift_all[:,1], second_deg_fit_80_100(res_delta_band_and_redshift_80_100[:,1].astype(float)), ls='--', color='blue', label='Brightest')
        plt.plot(res_delta_band_and_redshift_all[:,1], second_deg_fit_60_80(res_delta_band_and_redshift_60_80[:,1].astype(float)), ls='--', color='green', label='Brighter')
        plt.plot(res_delta_band_and_redshift_all[:,1], second_deg_fit_40_60(res_delta_band_and_redshift_40_60[:,1].astype(float)), ls='--', color='brown', label='Average')
        plt.plot(res_delta_band_and_redshift_all[:,1], second_deg_fit_20_40(res_delta_band_and_redshift_20_40[:,1].astype(float)), ls='--', color='orange', label='Dimmer')
        plt.plot(res_delta_band_and_redshift_all[:,1], second_deg_fit_0_20(res_delta_band_and_redshift_0_20[:,1].astype(float)), ls='--', color='red', label='Dimmest')        
        plt.ylabel('$\Delta$ BAND-band (SDSS - HSC)')
        plt.xlabel('redshift')
        if (ylim != None):
            plt.ylim(ylim[0], ylim[1])
            plt.xlim(0.1, 4)
        plt.legend()
        plt.show()

    elif (smoothing_method == 'median'):
        # PLotting
        matplotlib.rcParams.update({'font.size': 12})
        plt.figure(figsize=(18,6))
        plt.title('negative $\Delta$ BAND-band values= fainter in HSC; positive $\Delta$ BAND-band values= brighter in HSC')
        plt.errorbar(res_delta_band_and_redshift_all_med[:,1], res_delta_band_and_redshift_all_med[:,0], yerr=res_delta_std_all[:,0], color='grey', alpha=.5)
        plt.plot(res_delta_band_and_redshift_all_med[:,1], second_deg_fit_80_100(res_delta_band_and_redshift_80_100_med[:,1].astype(float)), ls='--', color='blue', label='Brightest')
        plt.plot(res_delta_band_and_redshift_all_med[:,1], second_deg_fit_60_80(res_delta_band_and_redshift_60_80_med[:,1].astype(float)), ls='--', color='green', label='Brighter')
        plt.plot(res_delta_band_and_redshift_all_med[:,1], second_deg_fit_40_60(res_delta_band_and_redshift_40_60_med[:,1].astype(float)), ls='--', color='brown', label='Average')
        plt.plot(res_delta_band_and_redshift_all_med[:,1], second_deg_fit_20_40(res_delta_band_and_redshift_20_40_med[:,1].astype(float)), ls='--', color='orange', label='Dimmer')
        plt.plot(res_delta_band_and_redshift_all_med[:,1], second_deg_fit_0_20(res_delta_band_and_redshift_0_20_med[:,1].astype(float)), ls='--', color='red', label='Dimmest')           
        plt.ylabel('$\Delta$ BAND-band (SDSS - HSC)')
        plt.xlabel('redshift')
        if (ylim != None):
            plt.ylim(ylim[0], ylim[1])
            plt.xlim(0.1, 4)
        else:
            pass
        plt.legend()
        plt.show()
    pass

In [39]:
# Test cell

DCSP(data_col=4, smoothing_method='median', ylim=(-.4, .4), num_to_choose=100)
#DCSP(data_col=18, smoothing_method='mean', ylim=(-.4,.4), num_to_choose=100)

## redshift effect 

In [ ]:
# array with has delta g as the first column, redshift as the second column
delta_g_and_redshift=matched_array_filtered_with_g_mag_dif_and_err[:,[4,3]]
delta_g_and_redshift_sorted_by_delta_g_and_split=np.array_split(delta_g_and_redshift[np.argsort(delta_g_and_redshift[:,0])],int(len(delta_g_and_redshift)/100))

# array with has delta r as the first column, redshift as the second column
delta_r_and_redshift=matched_array_filtered_with_r_mag_dif_and_err[:,[6,3]]

# array with has delta i as the first column, redshift as the second column
delta_i_and_redshift=matched_array_filtered_with_i_mag_dif_and_err[:,[8,3]]


# array with has delta gcmodel as the first column, redshift as the second column
delta_gcmodel_and_redshift=matched_array_filtered_with_gcmodel_mag_dif_and_err[:,[4,3]]

# array with has delta rcmodel as the first column, redshift as the second column
delta_rcmodel_and_redshift=matched_array_filtered_with_rcmodel_mag_dif_and_err[:,[6,3]]

# array with has delta icmodel as the first column, redshift as the second column
delta_icmodel_and_redshift=matched_array_filtered_with_icmodel_mag_dif_and_err[:,[8,3]]

In [ ]:
# previous array (g), sorted by delta g first and then split in bins, 100 objects in each bin
delta_g_and_redshift_sorted_by_delta_g_and_split=np.array_split(delta_g_and_redshift[np.argsort(delta_g_and_redshift[:,0])],int(len(delta_g_and_redshift)/100))
# previous array (g), sorted by redshift first and then split in bins, 100 objects in each bin
delta_g_and_redshift_sorted_by_redshift_g_and_split=np.array_split(delta_g_and_redshift[np.argsort(delta_g_and_redshift[:,1])],int(len(delta_g_and_redshift)/100))

# previous array (r), sorted by delta r first and then split in bins, 10 objects in each bin
delta_r_and_redshift_sorted_by_delta_r_and_split=np.array_split(delta_r_and_redshift[np.argsort(delta_r_and_redshift[:,0])],int(len(delta_r_and_redshift)/10))
# previous array (r), sorted by redshift first and then split in bins, 10 objects in each bin
delta_r_and_redshift_sorted_by_redshift_r_and_split=np.array_split(delta_r_and_redshift[np.argsort(delta_r_and_redshift[:,1])],int(len(delta_r_and_redshift)/10))

# previous array (i), sorted by delta i first and then split in bins, 10 objects in each bin
delta_i_and_redshift_sorted_by_delta_i_and_split=np.array_split(delta_i_and_redshift[np.argsort(delta_i_and_redshift[:,0])],int(len(delta_i_and_redshift)/10))
# previous array (i), sorted by redshift first and then split in bins, 10 objects in each bin
delta_i_and_redshift_sorted_by_redshift_i_and_split=np.array_split(delta_i_and_redshift[np.argsort(delta_i_and_redshift[:,1])],int(len(delta_i_and_redshift)/10))



# previous array (gcmodel), sorted by delta gmodel first and then split in bins, 10 objects in each bin
delta_gcmodel_and_redshift_sorted_by_delta_gcmodel_and_split=np.array_split(delta_gcmodel_and_redshift[np.argsort(delta_gcmodel_and_redshift[:,0])],int(len(delta_gcmodel_and_redshift)/10))
# previous array (gcmodel), sorted by redshift first and then split in bins, 10 objects in each bin
delta_gcmodel_and_redshift_sorted_by_redshift_gcmodel_and_split=np.array_split(delta_gcmodel_and_redshift[np.argsort(delta_gcmodel_and_redshift[:,1])],int(len(delta_gcmodel_and_redshift)/10))

# previous array (rcmodel), sorted by delta rcmodel first and then split in bins, 10 objects in each bin
delta_rcmodel_and_redshift_sorted_by_delta_rcmodel_and_split=np.array_split(delta_rcmodel_and_redshift[np.argsort(delta_rcmodel_and_redshift[:,0])],int(len(delta_rcmodel_and_redshift)/10))
# previous array (rcmodel), sorted by redshift first and then split in bins, 10 objects in each bin
delta_rcmodel_and_redshift_sorted_by_redshift_rcmodel_and_split=np.array_split(delta_rcmodel_and_redshift[np.argsort(delta_rcmodel_and_redshift[:,1])],int(len(delta_rcmodel_and_redshift)/10))

# previous array (icmodel), sorted by delta icmodel first and then split in bins, 10 objects in each bin
delta_icmodel_and_redshift_sorted_by_delta_icmodel_and_split=np.array_split(delta_icmodel_and_redshift[np.argsort(delta_icmodel_and_redshift[:,0])],int(len(delta_icmodel_and_redshift)/10))
# previous array (icmodel), sorted by redshift first and then split in bins, 10 objects in each bin
delta_icmodel_and_redshift_sorted_by_redshift_icmodel_and_split=np.array_split(delta_icmodel_and_redshift[np.argsort(delta_icmodel_and_redshift[:,1])],int(len(delta_icmodel_and_redshift)/10))

In [ ]:
# Smoothing the R-band data
res_delta_redshift_via_delta_r=[]
res_delta_redshift_via_delta_median_r=[]
for i in range(len(delta_r_and_redshift_sorted_by_delta_r_and_split)):
    res_delta_redshift_via_delta_r.append(np.mean(delta_r_and_redshift_sorted_by_delta_r_and_split[i],axis=0))
    res_delta_redshift_via_delta_median_r.append(np.median(delta_r_and_redshift_sorted_by_delta_r_and_split[i],axis=0))
    
res_delta_redshift_via_delta_r=np.array(res_delta_redshift_via_delta_r)
res_delta_redshift_via_delta_median_r=np.array(res_delta_redshift_via_delta_median_r)

res_delta_redshift_via_redshift_r=[]
res_delta_redshift_via_redshift_median_r=[]
for i in range(len(delta_r_and_redshift_sorted_by_redshift_r_and_split)):
    res_delta_redshift_via_redshift_r.append(np.mean(delta_r_and_redshift_sorted_by_redshift_r_and_split[i],axis=0))
    res_delta_redshift_via_redshift_median_r.append(np.median(delta_r_and_redshift_sorted_by_redshift_r_and_split[i],axis=0))
    
res_delta_redshift_via_redshift_r=np.array(res_delta_redshift_via_redshift_r)
res_delta_redshift_via_redshift_median_r=np.array(res_delta_redshift_via_redshift_median_r)

# Smoothing the I-band data
res_delta_redshift_via_delta_i=[]
res_delta_redshift_via_delta_median_i=[]
for i in range(len(delta_i_and_redshift_sorted_by_delta_i_and_split)):
    res_delta_redshift_via_delta_i.append(np.mean(delta_i_and_redshift_sorted_by_delta_i_and_split[i],axis=0))
    res_delta_redshift_via_delta_median_i.append(np.median(delta_i_and_redshift_sorted_by_delta_i_and_split[i],axis=0))
    
res_delta_redshift_via_delta_i=np.array(res_delta_redshift_via_delta_i)
res_delta_redshift_via_delta_median_i=np.array(res_delta_redshift_via_delta_median_i)

res_delta_redshift_via_redshift_i=[]
res_delta_redshift_via_redshift_median_i=[]
for i in range(len(delta_i_and_redshift_sorted_by_redshift_i_and_split)):
    res_delta_redshift_via_redshift_i.append(np.mean(delta_i_and_redshift_sorted_by_redshift_i_and_split[i],axis=0))
    res_delta_redshift_via_redshift_median_i.append(np.median(delta_i_and_redshift_sorted_by_redshift_i_and_split[i],axis=0))
    
res_delta_redshift_via_redshift_i=np.array(res_delta_redshift_via_redshift_i)
res_delta_redshift_via_redshift_median_i=np.array(res_delta_redshift_via_redshift_median_i)

# Smoothing the G-band data
res_delta_redshift_via_delta=[]
res_delta_redshift_via_delta_median=[]
for i in range(len(delta_g_and_redshift_sorted_by_delta_g_and_split)):
    res_delta_redshift_via_delta.append(np.mean(delta_g_and_redshift_sorted_by_delta_g_and_split[i],axis=0))
    res_delta_redshift_via_delta_median.append(np.median(delta_g_and_redshift_sorted_by_delta_g_and_split[i],axis=0))
    
res_delta_redshift_via_delta=np.array(res_delta_redshift_via_delta)
res_delta_redshift_via_delta_median=np.array(res_delta_redshift_via_delta_median)

res_delta_redshift_via_redshift=[]
res_delta_redshift_via_redshift_median=[]
for i in range(len(delta_g_and_redshift_sorted_by_redshift_g_and_split)):
    res_delta_redshift_via_redshift.append(np.mean(delta_g_and_redshift_sorted_by_redshift_g_and_split[i],axis=0))
    res_delta_redshift_via_redshift_median.append(np.median(delta_g_and_redshift_sorted_by_redshift_g_and_split[i],axis=0))
    
res_delta_redshift_via_redshift=np.array(res_delta_redshift_via_redshift)
res_delta_redshift_via_redshift_median=np.array(res_delta_redshift_via_redshift_median)


# Further smoothing. We choose, out of the 100 objects in each bin, the brightest 20 and the dimmest 20.
# res_delta_redshift_via_redshift_min=[]
# res_delta_redshift_via_redshift_max=[]
# for i in range(len(delta_g_and_redshift_sorted_by_redshift_g_and_split)):
#     res_delta_redshift_via_redshift_min.append(delta_g_and_redshift_sorted_by_redshift_g_and_split[i].astype(float).sort()[-20:])
#     res_delta_redshift_via_redshift_max.append(delta_g_and_redshift_sorted_by_redshift_g_and_split[i].sort()[:20])

In [ ]:
# Same thing, but for the cmodel data.
res_delta_redshift_via_delta_rcmodel=[]
res_delta_redshift_via_delta_median_rcmodel=[]
for i in range(len(delta_rcmodel_and_redshift_sorted_by_delta_rcmodel_and_split)):
    res_delta_redshift_via_delta_rcmodel.append(np.mean(delta_rcmodel_and_redshift_sorted_by_delta_rcmodel_and_split[i],axis=0))
    res_delta_redshift_via_delta_median_rcmodel.append(np.median(delta_rcmodel_and_redshift_sorted_by_delta_rcmodel_and_split[i],axis=0))
    
res_delta_redshift_via_delta_rcmodel=np.array(res_delta_redshift_via_delta_rcmodel)
res_delta_redshift_via_delta_median_rcmodel=np.array(res_delta_redshift_via_delta_median_rcmodel)

res_delta_redshift_via_redshift_rcmodel=[]
res_delta_redshift_via_redshift_median_rcmodel=[]
for i in range(len(delta_rcmodel_and_redshift_sorted_by_redshift_rcmodel_and_split)):
    res_delta_redshift_via_redshift_rcmodel.append(np.mean(delta_rcmodel_and_redshift_sorted_by_redshift_rcmodel_and_split[i],axis=0))
    res_delta_redshift_via_redshift_median_rcmodel.append(np.median(delta_rcmodel_and_redshift_sorted_by_redshift_rcmodel_and_split[i],axis=0))
    
res_delta_redshift_via_redshift_rcmodel=np.array(res_delta_redshift_via_redshift_rcmodel)
res_delta_redshift_via_redshift_median_rcmodel=np.array(res_delta_redshift_via_redshift_median_rcmodel)

res_delta_redshift_via_delta_icmodel=[]
res_delta_redshift_via_delta_median_icmodel=[]
for i in range(len(delta_icmodel_and_redshift_sorted_by_delta_icmodel_and_split)):
    res_delta_redshift_via_delta_icmodel.append(np.mean(delta_icmodel_and_redshift_sorted_by_delta_icmodel_and_split[i],axis=0))
    res_delta_redshift_via_delta_median_icmodel.append(np.median(delta_icmodel_and_redshift_sorted_by_delta_icmodel_and_split[i],axis=0))
    
res_delta_redshift_via_delta_icmodel=np.array(res_delta_redshift_via_delta_icmodel)
res_delta_redshift_via_delta_median_icmodel=np.array(res_delta_redshift_via_delta_median_icmodel)

res_delta_redshift_via_redshift_icmodel=[]
res_delta_redshift_via_redshift_median_icmodel=[]
for i in range(len(delta_icmodel_and_redshift_sorted_by_redshift_icmodel_and_split)):
    res_delta_redshift_via_redshift_icmodel.append(np.mean(delta_icmodel_and_redshift_sorted_by_redshift_icmodel_and_split[i],axis=0))
    res_delta_redshift_via_redshift_median_icmodel.append(np.median(delta_icmodel_and_redshift_sorted_by_redshift_icmodel_and_split[i],axis=0))
    
res_delta_redshift_via_redshift_icmodel=np.array(res_delta_redshift_via_redshift_icmodel)
res_delta_redshift_via_redshift_median_icmodel=np.array(res_delta_redshift_via_redshift_median_icmodel)

res_delta_redshift_via_delta_gcmodel=[]
res_delta_redshift_via_delta_median_gcmodel=[]
for i in range(len(delta_gcmodel_and_redshift_sorted_by_delta_gcmodel_and_split)):
    res_delta_redshift_via_delta_gcmodel.append(np.mean(delta_gcmodel_and_redshift_sorted_by_delta_gcmodel_and_split[i],axis=0))
    res_delta_redshift_via_delta_median_gcmodel.append(np.median(delta_gcmodel_and_redshift_sorted_by_delta_gcmodel_and_split[i],axis=0))
    
res_delta_redshift_via_delta_gcmodel=np.array(res_delta_redshift_via_delta_gcmodel)
res_delta_redshift_via_delta_median_gcmodel=np.array(res_delta_redshift_via_delta_median_gcmodel)

res_delta_redshift_via_redshift_gcmodel=[]
res_delta_redshift_via_redshift_median_gcmodel=[]
for i in range(len(delta_gcmodel_and_redshift_sorted_by_redshift_gcmodel_and_split)):
    res_delta_redshift_via_redshift_gcmodel.append(np.mean(delta_gcmodel_and_redshift_sorted_by_redshift_gcmodel_and_split[i],axis=0))
    res_delta_redshift_via_redshift_median_gcmodel.append(np.median(delta_gcmodel_and_redshift_sorted_by_redshift_gcmodel_and_split[i],axis=0))
    
res_delta_redshift_via_redshift_gcmodel=np.array(res_delta_redshift_via_redshift_gcmodel)
res_delta_redshift_via_redshift_median_gcmodel=np.array(res_delta_redshift_via_redshift_median_gcmodel)

In [ ]:
p20_g=np.poly1d(np.polyfit(res_delta_redshift_via_redshift[:,1].astype(float),res_delta_redshift_via_redshift[:,0].astype(float),2))
p20_r=np.poly1d(np.polyfit(res_delta_redshift_via_redshift_r[:,1].astype(float),res_delta_redshift_via_redshift_r[:,0].astype(float),2))
p20_i=np.poly1d(np.polyfit(res_delta_redshift_via_redshift_i[:,1].astype(float),res_delta_redshift_via_redshift_i[:,0].astype(float),2))

p20_gcmodel=np.poly1d(np.polyfit(res_delta_redshift_via_redshift_gcmodel[:,1].astype(float),res_delta_redshift_via_redshift_gcmodel[:,0].astype(float),2))
p20_rcmodel=np.poly1d(np.polyfit(res_delta_redshift_via_redshift_rcmodel[:,1].astype(float),res_delta_redshift_via_redshift_rcmodel[:,0].astype(float),2))
p20_icmodel=np.poly1d(np.polyfit(res_delta_redshift_via_redshift_icmodel[:,1].astype(float),res_delta_redshift_via_redshift_icmodel[:,0].astype(float),2))

# Plots

## Change in luminosity as a function of redshift. HSC data is from psf model.

In [ ]:
# below we are seeing, most probably, difference between SDSS and HSC filters 
matplotlib.rcParams.update({'font.size': 12})
plt.figure(figsize=(18,6))
plt.subplot(121)
plt.suptitle('negative $\Delta$ g-band values= fainter in HSC; positive $\Delta$ g-band values= brighter in HSC; Both data from psf')
plt.plot(res_delta_redshift_via_delta[:,0],res_delta_redshift_via_delta[:,1])
#plt.plot(res_delta_redshift_via_delta_median[:,0],res_delta_redshift_via_delta_median[:,1],alpha=0.2)
plt.xlabel('$\Delta$ g-band (SDSS - HSC)')
plt.ylabel('redshift')
plt.subplot(122)
plt.plot(res_delta_redshift_via_redshift[:,1],res_delta_redshift_via_redshift[:,0])
plt.plot(res_delta_redshift_via_redshift[:,1],p20_g(res_delta_redshift_via_redshift[:,1].astype(float)),ls='--',color='orange')
#plt.plot(res_delta_redshift_via_redshift_median[:,1],res_delta_redshift_via_redshift_median[:,0],alpha=0.2)
plt.ylabel('$\Delta$ g-band (SDSS - HSC)')
plt.xlabel('redshift')
plt.show()

In [ ]:
matplotlib.rcParams.update({'font.size': 12})
plt.figure(figsize=(18,6))
plt.subplot(121)
plt.suptitle('negative $\Delta$ r-band values= fainter in HSC; positive $\Delta$ r-band values= brighter in HSC; Both data from psf')
plt.plot(res_delta_redshift_via_delta_r[:,0],res_delta_redshift_via_delta_r[:,1])
#plt.plot(res_delta_redshift_via_delta_median[:,0],res_delta_redshift_via_delta_median[:,1],alpha=0.2)
plt.xlabel('$\Delta$ g-band (SDSS - HSC)')
plt.ylabel('redshift')
plt.subplot(122)
plt.plot(res_delta_redshift_via_redshift_r[:,1],res_delta_redshift_via_redshift_r[:,0])
plt.plot(res_delta_redshift_via_redshift_r[:,1],p20_r(res_delta_redshift_via_redshift_r[:,1]),ls='--',color='orange')
#plt.plot(res_delta_redshift_via_redshift_median[:,1],res_delta_redshift_via_redshift_median[:,0],alpha=0.2)
plt.ylabel('$\Delta$ g-band (SDSS - HSC)')
plt.xlabel('redshift')
plt.show()

In [ ]:
matplotlib.rcParams.update({'font.size': 12})
plt.figure(figsize=(18,6))
plt.subplot(121)
plt.suptitle('negative $\Delta$ i-band values= fainter in HSC; positive $\Delta$ i-band values= brighter in HSC; Both data from psf')
plt.plot(res_delta_redshift_via_delta_i[:,0],res_delta_redshift_via_delta_i[:,1])
#plt.plot(res_delta_redshift_via_delta_median[:,0],res_delta_redshift_via_delta_median[:,1],alpha=0.2)
plt.xlabel('$\Delta$ i-band (SDSS - HSC)')
plt.ylabel('redshift')
plt.subplot(122)
plt.plot(res_delta_redshift_via_redshift_i[:,1],res_delta_redshift_via_redshift_i[:,0])
plt.plot(res_delta_redshift_via_redshift_i[:,1],p20_i(res_delta_redshift_via_redshift_i[:,1]),ls='--',color='orange')
#plt.plot(res_delta_redshift_via_redshift_median[:,1],res_delta_redshift_via_redshift_median[:,0],alpha=0.2)
plt.ylabel('$\Delta$ i-band (SDSS - HSC)')
plt.xlabel('redshift')
plt.show()

## Change in luminosity as a function of redshift. HSC data is from cmodel.

In [ ]:
# below we are seeing, most probably, difference between SDSS and HSC filters 
matplotlib.rcParams.update({'font.size': 12})
plt.figure(figsize=(18,6))
plt.subplot(121)
plt.suptitle('negative $\Delta$ g-band values= fainter in HSC; positive $\Delta$ g-band values= brighter in HSC; HSC data is from cmodel')
plt.plot(res_delta_redshift_via_delta_gcmodel[:,0],res_delta_redshift_via_delta_gcmodel[:,1])
#plt.plot(res_delta_redshift_via_delta_median[:,0],res_delta_redshift_via_delta_median[:,1],alpha=0.2)
plt.xlabel('$\Delta$ g-band (SDSS - HSC)')
plt.ylabel('redshift')
plt.subplot(122)
plt.plot(res_delta_redshift_via_redshift_gcmodel[:,1],res_delta_redshift_via_redshift_gcmodel[:,0])
plt.plot(res_delta_redshift_via_redshift_gcmodel[:,1],p20_g(res_delta_redshift_via_redshift_gcmodel[:,1].astype(float)),ls='--',color='orange')
#plt.plot(res_delta_redshift_via_redshift_median[:,1],res_delta_redshift_via_redshift_median[:,0],alpha=0.2)
plt.ylabel('$\Delta$ g-band (SDSS - HSC)')
plt.xlabel('redshift')
plt.show()

In [ ]:
matplotlib.rcParams.update({'font.size': 12})
plt.figure(figsize=(18,6))
plt.subplot(121)
plt.suptitle('negative $\Delta$ r-band values= fainter in HSC; positive $\Delta$ r-band values= brighter in HSC; Both data from psf')
plt.plot(res_delta_redshift_via_delta_rcmodel[:,0],res_delta_redshift_via_delta_rcmodel[:,1])
#plt.plot(res_delta_redshift_via_delta_median[:,0],res_delta_redshift_via_delta_median[:,1],alpha=0.2)
plt.xlabel('$\Delta$ g-band (SDSS - HSC)')
plt.ylabel('redshift')
plt.subplot(122)
plt.plot(res_delta_redshift_via_redshift_rcmodel[:,1],res_delta_redshift_via_redshift_rcmodel[:,0])
plt.plot(res_delta_redshift_via_redshift_rcmodel[:,1],p20_r(res_delta_redshift_via_redshift_rcmodel[:,1]),ls='--',color='orange')
#plt.plot(res_delta_redshift_via_redshift_median[:,1],res_delta_redshift_via_redshift_median[:,0],alpha=0.2)
plt.ylabel('$\Delta$ g-band (SDSS - HSC)')
plt.xlabel('redshift')
plt.show()

In [ ]:
matplotlib.rcParams.update({'font.size': 12})
plt.figure(figsize=(18,6))
plt.subplot(121)
plt.suptitle('negative $\Delta$ i-band values= fainter in HSC; positive $\Delta$ i-band values= brighter in HSC; HSC data is from cmodel')
plt.plot(res_delta_redshift_via_delta_icmodel[:,0],res_delta_redshift_via_delta_icmodel[:,1])
#plt.plot(res_delta_redshift_via_delta_median[:,0],res_delta_redshift_via_delta_median[:,1],alpha=0.2)
plt.xlabel('$\Delta$ i-band (SDSS - HSC)')
plt.ylabel('redshift')
plt.subplot(122)
plt.plot(res_delta_redshift_via_redshift_icmodel[:,1],res_delta_redshift_via_redshift_icmodel[:,0])
plt.plot(res_delta_redshift_via_redshift_icmodel[:,1],p20_i(res_delta_redshift_via_redshift_icmodel[:,1]),ls='--',color='orange')
#plt.plot(res_delata_redshift_via_redshift_median[:,1],res_delta_redshift_via_redshift_median[:,0],alpha=0.2)
plt.ylabel('$\Delta$ i-band (SDSS - HSC)')
plt.xlabel('redshift')
plt.show()

## More graphs

In [ ]:
res_delta_redshift_via_redshift

In [ ]:
matplotlib.rcParams.update({'font.size': 12})
plt.figure(figsize=(18,6))
plt.plot(res_delta_redshift_via_redshift[:,1],res_delta_redshift_via_redshift_median[:,0])
plt.plot(res_delta_redshift_via_redshift[:,1],p20_g(res_delta_redshift_via_redshift[:,1].astype(float)),ls='--',color='orange')
plt.ylabel('$\Delta$ g-band (SDSS - HSC)')
plt.xlabel('redshift')
plt.show()

## dependence with Mass, Luminosity, Eddington ratio

## Nice plot summarizing what we've done.

In [ ]:
from astropy.io import fits

dr7_bh=fits.open('/Users/nevencaplar/Documents/Variability/HSC_Data/dr7_bh_May09_2011.fits')

# Star test

In [32]:
test = pd.read_csv('~/stripe82calibStars_v2.6.dat', header=None, delimiter='\s+', names=['Useless', 'RA', 'DEC', 'RA_rms', 'DEC_rms', 'Ntot', 'Ar', 'u_obs_num', 'u_median_mag', 'u_mean_mag', 'u_mean_err', 'u_rms', 'u_chi2', 'g_obs_num', 'g_median_mag', 'g_mean_mag', 'g_mean_err', 'g_rms', 'g_chi2', 'r_obs_num', 'r_median_mag', 'r_mean_mag', 'r_mean_err', 'r_rms', 'r_chi2', 'i_obs_num', 'i_median_mag', 'i_mean_mag', 'i_mean_err', 'i_rms', 'i_chi2', 'z_obs_num', 'z_median_mag', 'z_mean_mag', 'z_mean_err', 'z_rms', 'z_chi2',], usecols=['RA', 'DEC', 'g_mean_mag'], skiprows=41, engine='python', dtype='f8')

In [33]:
test.iloc[0,:]

RA            308.500214
DEC            -1.227721
g_mean_mag     22.265000
Name: 0, dtype: float64

In [70]:
star_coords_and_gmag

array([[ 3.08500214e+02, -1.22772100e+00,  2.22650000e+01],
       [ 3.08500092e+02, -1.24026420e+00,  2.04470000e+01],
       [ 3.08500092e+02, -1.21715590e+00,  1.80000000e+01],
       ...,
       [ 5.99976196e+01, -4.10360000e-02,  2.18730000e+01],
       [ 5.99978180e+01, -9.27300000e-02,  2.27390000e+01],
       [ 5.99975166e+01,  1.07724020e+00,  1.49870000e+01]])

In [81]:
star_coords_and_gmag[star_coords_and_gmag[:,2]>22.5].shape

(99817, 3)

In [69]:
star_coords_and_gmag = test.values
star_coords_and_gmag.shape

# Mask out irrelevant stars. https://stackoverflow.com/questions/47819146/how-to-delete-a-row-based-on-a-condition-from-a-numpy-array
mask = (star_coords_and_gmag <= 50).all(axis=)
star_coords_and_gmag = star_coords_and_gmag[mask, :]
print(star_coords_and_gmag.shape)

AxisError: axis 2 is out of bounds for array of dimension 2

In [57]:
SQL_string = ""

In [58]:
for i in range(star_coords_and_gmag.shape[0]):
    SQL_string += "(" + str(star_coords_and_gmag[i][0]) + "," + str(star_coords_and_gmag[i][1]) + "), "

In [59]:
with open("SQL_stuff.txt", "w") as text_file:
    text_file.write(SQL_string)